In [2]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
match_sum = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/STATS_S02.xlsx', 
                         sheet_name='Match_Summary')

In [4]:
matches_list1 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_01')
matches_list2 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_02')

matches_list1['match_id'] = matches_list1['num'].apply(lambda x: 'M00' + str(x) if x<=9 else 'M0' + str(x))
matches_list2['match_id'] = matches_list2['num'].apply(lambda x: 'S02M00' + str(x) if x<=9 else 'S02M0' + str(x))

matches_list = pd.concat([matches_list1[['num','Spin','match_id']],matches_list2[['num','Spin','match_id']]],
                         axis=0)



matches_list.columns
#matches_list = matches_list[['Date','Team One','Team Two']]

Index(['num', 'Spin', 'match_id'], dtype='object')

matches_list

In [5]:
#df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats//df_all_round_sim.csv')
df_sim2 = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')

#df_sim = pd.concat([df_sim,df_sim2], axis=0).reset_index(drop=True)
df_sim = df_sim2

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_01 = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats/df_all_round_sim.csv')
df_curr = pd.DataFrame(columns=df_01.columns.tolist())
try:
    df_curr = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')
    #df_curr = df_curr[df_01.columns.tolist()]
except FileNotFoundError:
    print("df_curr file not found, proceeding with df_01 only.")

df_all = pd.concat([df_01,df_curr], axis=0)
df_all.reset_index(drop=True, inplace=True)


df_all['phase'] = np.where(df_all['legal_balls_bowled']<=36, 'pp', 
                        np.where(df_all['legal_balls_bowled']>=90, 'death',
                            'middle'))

df_sim = df_all

([1, 2, 6, 1, 6, 1, 2, 4, 3, 4, 3, 4, 3, 4, 3, 5, 2, 5, 2, 5],
 [1, 5, 1, 6, 5, 1, 6, 3, 4, 6, 3, 6, 3, 4, 3, 2, 5, 2, 5, 2])

df_sim[df_sim['batting_team']=='CSK']['striker'].unique()

In [7]:
play_style_df = pd.read_csv("/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/play_style.csv")


In [8]:
df_sim.loc[df_sim.isWicket==1, 'partnership_runs'] = np.nan

df_sim['partnership_runs'] = df_sim['partnership_runs'].fillna(method='ffill')
df_sim.loc[df_sim.isWicket==1, 'partnership_runs'] = df_sim.loc[df_sim.isWicket==1, 'partnership_runs']+\
                                            df_sim.loc[df_sim.isWicket==1, 'total_runs']

In [9]:
#samp = df_sim[df_sim.match_id=='M072']

df_sim['before_ball_partnership'] = 0
df_sim['after_ball_partnership'] = 0
df_sim['before_balls'] = 0
df_sim['after_balls'] = 0

for m_id in df_sim.match_id.unique():
    #print(m_id)
    bp = 0
    ap = 0
    bb = 0
    ab = 0
    for inn in df_sim[df_sim.match_id==m_id].innings.unique():
        bp = 0
        ap = 0
        bb = 0
        ab = 0
        for index,row in df_sim[df_sim.match_id==m_id][df_sim.innings==inn].iterrows():
            ap = bp + row['total_runs']
            ab = bb + row['is_faced_by_batter']
            #print(ap,bp)
            df_sim.at[index, 'before_ball_partnership'] = bp
            df_sim.at[index, 'after_ball_partnership'] = ap
            df_sim.at[index, 'before_balls'] = bb
            df_sim.at[index, 'after_balls'] = ab
            bp = ap
            bb = ab
            if row['isWicket']==1:
                bp = 0
                ap = 0
                bb = 0
                ab = 0
        

p_sub = df_sim[['wickets_down','after_ball_partnership']]\
                .sort_values(['wickets_down','after_ball_partnership'], ascending=[True,False])

p_df = p_sub.groupby('wickets_down')['after_ball_partnership'].head(5).reset_index()

#p_df

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3200/1875421427.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for index,row in df_sim[df_sim.match_id==m_id][df_sim.innings==inn].iterrows():


In [10]:
df_sim['over_1'] = df_sim['legal_balls_bowled']//6
df_sim['over_2'] = df_sim['legal_balls_bowled']%6
df_sim['over_1'] = df_sim['over_1'].astype(str)
df_sim['over_2'] = df_sim['over_2'].astype(str)

df_sim['over'] = df_sim['over_1'] + "." +  df_sim['over_2']
df_sim['over'] = df_sim['over'].astype(float)

df_sim.drop(['over_1','over_2'],axis=1,inplace=True)
df_sim['over_'] = np.ceil(df_sim['over'])
df_sim['over_'] = df_sim['over_'].astype(int)

df_sim['phase'] = np.where(df_sim['over']<=6, "PP", np.where(df_sim['over']<=15, "Middle", "Death"))

In [11]:
df_sim = df_sim.merge(matches_list[['match_id','Spin']], on='match_id', how='left')
df_sim['pitch'] = df_sim['Spin'].apply(lambda x: 'pace' if x<=40 
                                       else 'spin' if x>=60
                                      else 'neutralish')

df_sim.sample(5)

df_sim.columns

df_sim[df_sim.season==2][df_sim.wickets_down.isin([0])][['match_id','venue','batting_team','bowling_team',
                'striker','non_striker','before_ball_partnership', 'after_ball_partnership',
       'before_balls', 'after_balls',
                    'runs_scored','legal_balls_bowled']]\
    .sort_values(['before_ball_partnership', 'after_ball_partnership'],
                 ascending=False).head(15)

df_sim[df_sim.season==2][df_sim.wickets_down.isin([1])][['match_id','venue','batting_team','bowling_team',
                'striker','non_striker','before_ball_partnership', 'after_ball_partnership',
       'before_balls', 'after_balls',
                    'runs_scored','legal_balls_bowled']]\
    .sort_values(['before_ball_partnership', 'after_ball_partnership'],
                 ascending=True).head(15)

In [12]:
df_sim = df_sim.merge(play_style_df, on='bowler',
                       how='left')

## spin-pace stats this season

In [13]:



spin_bat = df_sim[df_sim['bowl_style_parent']=='Spin'].groupby('striker').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

spin_bat['SR'] = 100*spin_bat['runs']/spin_bat['balls']
spin_bat['Bat_avg'] = spin_bat['runs']/spin_bat['outs']
spin_bat['bpb'] = spin_bat['balls']/(spin_bat['fours']+spin_bat['sixes'])
spin_bat['dot_%'] = 100*spin_bat['dots']/spin_bat['balls']

pace_bat = df_sim[df_sim['bowl_style_parent']=='Pace'].groupby('striker').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

pace_bat['SR'] = 100*pace_bat['runs']/pace_bat['balls']
pace_bat['Bat_avg'] = pace_bat['runs']/pace_bat['outs']
pace_bat['bpb'] = pace_bat['balls']/(pace_bat['fours']+pace_bat['sixes'])
pace_bat['dot_%'] = 100*pace_bat['dots']/pace_bat['balls']
pace_bat['bowling_type'] = 'pace'
pace_bat['pace_index_int'] = pace_bat['SR']+np.where(pace_bat['Bat_avg']<1000, pace_bat['Bat_avg'], \
                                pace_bat['runs'])- 10*np.where(pace_bat['bpb']<1000, pace_bat['bpb'], \
                                        pace_bat['balls'])-pace_bat['dot_%']

pace_bat['pace_idx'] = np.where(
    ((pace_bat['balls'] < 300) ), #& (pace_bat['balls'] >= 50) 
    pace_bat['pace_index_int'] * ((pace_bat['balls'] / 300) ** 0.5),\
    np.where(pace_bat['balls'] >= 300, 
    pace_bat['pace_index_int'], \
            np.nan)
)


##
spin_bat['spin_index_int'] = spin_bat['SR']+np.where(spin_bat['Bat_avg']<1000, spin_bat['Bat_avg'], \
                                spin_bat['runs'])- 10*np.where(spin_bat['bpb']<1000, spin_bat['bpb'], \
                                        spin_bat['balls'])-spin_bat['dot_%']
spin_bat['bowling_type'] = 'spin'
spin_bat['spin_idx'] = np.where(
    ((spin_bat['balls'] < 300) ),  #& (spin_bat['balls'] >= 50) 
    spin_bat['spin_index_int'] * ((spin_bat['balls'] / 300) ** 0.5),\
    np.where(spin_bat['balls'] >= 300, 
    spin_bat['spin_index_int'], \
            np.nan)
)

##

k1 = 0.02
k2 = 0.1
c1 = 70
c2 = 97

def sigmoid(x):
    if x<=70:
        value = 0.5 + 1 / (1 + np.exp(-k1 * (x-c1)))
    else:
        value = 1 + 1/ (1 + np.exp(-k2 * (x-c2 )))
        
    return value

pace_bat['pace_index'] = pace_bat['pace_idx'].apply(sigmoid)
spin_bat['spin_index'] = spin_bat['spin_idx'].apply(sigmoid)

##
pace_bat.drop(['pace_index_int','pace_idx'], axis=1, inplace=True)
spin_bat.drop(['spin_index_int','spin_idx'], axis=1, inplace=True)



pl_st_df = pd.concat([pace_bat,spin_bat], axis=0)
pl_st_df = pl_st_df.round(2)

In [52]:
str_ = 'Stokes'
pl_st_df[pl_st_df['striker'].str.contains(str_)]\
        .sort_values('striker',ascending=False)

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,bowling_type,pace_index,spin_index
24,BA Stokes,291,211,15,29,12,92,137.91,19.40,5.15,43.60,pace,0.91,NaN
26,BA Stokes,202,127,6,18,9,44,159.06,33.67,4.70,34.65,spin,NaN,1.08


## spin-pace stats for each team

In [726]:
df_sim = df_sim.merge(play_style_df, on='bowler',
                       how='left')

In [727]:
spin_bat = df_sim[df_sim['bowl_style_parent']=='Spin'].groupby('batting_team').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

spin_bat['SR'] = 100*spin_bat['runs']/spin_bat['balls']
spin_bat['Bat_avg'] = spin_bat['runs']/spin_bat['outs']
spin_bat['bpb'] = spin_bat['balls']/(spin_bat['fours']+spin_bat['sixes'])
spin_bat['dot_%'] = 100*spin_bat['dots']/spin_bat['balls']

pace_bat = df_sim[df_sim['bowl_style_parent']=='Pace'].groupby('batting_team').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

pace_bat['SR'] = 100*pace_bat['runs']/pace_bat['balls']
pace_bat['Bat_avg'] = pace_bat['runs']/pace_bat['outs']
pace_bat['bpb'] = pace_bat['balls']/(pace_bat['fours']+pace_bat['sixes'])
pace_bat['dot_%'] = 100*pace_bat['dots']/pace_bat['balls']
pace_bat['bowling_type'] = 'pace'
pace_bat['pace_index_int'] = pace_bat['SR']+np.where(pace_bat['Bat_avg']<1000, pace_bat['Bat_avg'], \
                                pace_bat['runs'])- 10*np.where(pace_bat['bpb']<1000, pace_bat['bpb'], \
                                        pace_bat['balls'])-pace_bat['dot_%']

pace_bat['pace_idx'] = np.where(
    ((pace_bat['balls'] < 300) ), #& (pace_bat['balls'] >= 50) 
    pace_bat['pace_index_int'] * ((pace_bat['balls'] / 300) ** 0.5),\
    np.where(pace_bat['balls'] >= 300, 
    pace_bat['pace_index_int'], \
            np.nan)
)


##
spin_bat['spin_index_int'] = spin_bat['SR']+np.where(spin_bat['Bat_avg']<1000, spin_bat['Bat_avg'], \
                                spin_bat['runs'])- 10*np.where(spin_bat['bpb']<1000, spin_bat['bpb'], \
                                        spin_bat['balls'])-spin_bat['dot_%']
spin_bat['bowling_type'] = 'spin'
spin_bat['spin_idx'] = np.where(
    ((spin_bat['balls'] < 300) ),  #& (spin_bat['balls'] >= 50) 
    spin_bat['spin_index_int'] * ((spin_bat['balls'] / 300) ** 0.5),\
    np.where(spin_bat['balls'] >= 300, 
    spin_bat['spin_index_int'], \
            np.nan)
)

##

k1 = 0.02
k2 = 0.1
c1 = 70
c2 = 97

def sigmoid(x):
    if x<=70:
        value = 0.5 + 1 / (1 + np.exp(-k1 * (x-c1)))
    else:
        value = 1 + 1/ (1 + np.exp(-k2 * (x-c2 )))
        
    return value

pace_bat['pace_index'] = pace_bat['pace_idx'].apply(sigmoid)
spin_bat['spin_index'] = spin_bat['spin_idx'].apply(sigmoid)

##
pace_bat.drop(['pace_index_int','pace_idx'], axis=1, inplace=True)
spin_bat.drop(['spin_index_int','spin_idx'], axis=1, inplace=True)



pl_st_df = pd.concat([pace_bat,spin_bat], axis=0)
pl_st_df = pl_st_df.round(2)

In [731]:
pl_st_df[pl_st_df.bowling_type=='spin']

,batting_team,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,bowling_type,pace_index,spin_index
0,CSK,1836,1361,66,159,66,484,134.90,27.82,6.05,35.56,spin,NaN,0.98
1,DC,2646,1764,78,186,125,560,150.00,33.92,5.67,31.75,spin,NaN,1.46
2,GT,2165,1422,51,163,87,410,152.25,42.45,5.69,28.83,spin,NaN,1.77
3,KKR,1629,1120,47,129,68,371,145.45,34.66,5.69,33.12,spin,NaN,1.33
4,LSG,2499,1714,77,190,113,594,145.80,32.45,5.66,34.66,spin,NaN,1.27
5,MI,1983,1363,61,164,68,415,145.49,32.51,5.88,30.45,spin,NaN,1.31
6,PBKS,2545,1770,90,193,114,594,143.79,28.28,5.77,33.56,spin,NaN,1.17
7,RCB,1910,1378,63,144,82,465,138.61,30.32,6.10,33.74,spin,NaN,1.09
8,RR,2670,1761,83,222,108,524,151.62,32.17,5.34,29.76,spin,NaN,1.59
9,SRH,3091,2081,79,232,148,657,148.53,39.13,5.48,31.57,spin,NaN,1.61


## position wise batting stats

In [39]:
bat_list = ['VR Iyer','T Stubbs','KL Rahul']
output_df = pd.DataFrame()

for bat in bat_list:
    ms1 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([1]))]['match_id'].unique()
    ms2 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([2]))]['match_id'].unique()
    ms3 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([3]))]['match_id'].unique()
    ms4 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([4]))]['match_id'].unique()
    ms5 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([5]))]['match_id'].unique()
    ms6 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([6]))]['match_id'].unique()
    ms7 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([7]))]['match_id'].unique()
    ms8 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([8]))]['match_id'].unique()
    ms9 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([9]))]['match_id'].unique()
    ms10 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([10]))]['match_id'].unique()
    ms11 = df_sim[(df_sim['striker']==bat)&(df_sim.striker_batting_position.isin([11]))]['match_id'].unique()


    m_list =[ms1,ms2,ms3,ms4,ms5,ms6,ms7,ms8,ms9,ms10,ms11]

    for m in m_list:
        bat_st = df_sim[df_sim['match_id'].isin(m)].groupby(['striker','striker_batting_position']).agg(
            runs = ('runs_off_bat','sum'),
            balls = ('is_faced_by_batter','sum'),
            fours = ('isFour','sum'),
            sixes = ('isSix','sum'),
            dots = ('isDotforBatter','sum')
        ).reset_index()

        out_df = df_sim[df_sim['match_id'].isin(m)].groupby('player_dismissed')['isWicket'].sum().reset_index()
        out_df.columns = ['striker','outs']

        bat_st = bat_st.merge(out_df, on='striker', how='left')

        bat_st['SR'] = 100*bat_st['runs']/bat_st['balls']
        bat_st['Bat_avg'] = bat_st['runs']/bat_st['outs']
        bat_st['bpb'] = bat_st['balls']/(bat_st['fours']+bat_st['sixes'])
        bat_st['dot_%'] = 100*bat_st['dots']/bat_st['balls']

        bat_st = bat_st.round(2)
        output = bat_st[bat_st.striker==bat]
        output_df = output_df.append(output)

output_df['outs'] = output_df['outs'].fillna(0).astype(int)

In [40]:
output_df

,striker_batting_position,runs,balls,fours,sixes,dots,striker,outs,SR,Bat_avg,bpb,dot_%
168,4,690,462,67,22,135,VR Iyer,14,149.35,49.29,5.19,29.22
44,5,94,49,9,5,12,VR Iyer,3,191.84,31.33,3.50,24.49
19,6,25,13,2,1,0,VR Iyer,0,192.31,NaN,4.33,0.00
51,4,314,197,24,14,51,T Stubbs,5,159.39,62.80,5.18,25.89
123,5,344,256,23,13,85,T Stubbs,9,134.38,38.22,7.11,33.20
41,1,494,333,43,22,109,KL Rahul,11,148.35,44.91,5.12,32.73
18,2,74,60,5,3,23,KL Rahul,4,123.33,18.50,7.50,38.33
20,3,229,153,18,9,36,KL Rahul,1,149.67,229.00,5.67,23.53
15,4,234,145,20,6,28,KL Rahul,1,161.38,234.00,5.58,19.31
12,5,92,53,7,4,10,KL Rahul,2,173.58,46.00,4.82,18.87


## wkt-wise partnership stats

In [689]:
def safe_subtract(a, b):
    if pd.isna(b):
        return np.nan
    if pd.isna(a):
        return np.nan
    return b - a

In [708]:
p_df_all = pd.DataFrame()

teams = ['CSK','DC','GT','LSG','KKR','MI','PBKS','RCB','RR','SRH']

for team in teams:
    wkt = 10


    partnership_df = pd.DataFrame(np.nan, index=range(14), columns= 
                                           ['team','Match','1st Wkt','2nd Wkt','3rd Wkt','4th Wkt','5th Wkt',
                                          '6th Wkt','7th Wkt','8th Wkt','9th Wkt','10th Wkt',
                                                   '1st balls','2nd balls','3rd balls','4th balls','5th balls',
                                          '6th balls','7th balls','8th balls','9th balls','10th balls',
                                           '1st out','2nd out','3rd out','4th out','5th out',
                                          '6th out','7th out','8th out','9th out','10th out'])


    #fow_list = []
    #fow_balls_ = []
    i = -1

    for m in df_sim[(df_sim.batting_team==team)&(df_sim.match_id.str.contains('S02'))]['match_id'].unique():
        #print(m)
        i+=1
        sub_df = df_sim[(df_sim.match_id==m)&(df_sim.wickets_down<=(wkt+1))&(df_sim.batting_team==team)]

        fow = sub_df[sub_df.isWicket==1]['last_fow'].astype(int).tolist()
        fow_rr = sub_df[sub_df.isWicket==1]['last_fow_run_rate'].tolist()
        fow_balls = sub_df[sub_df.isWicket==1]['legal_balls_bowled'].astype(int).tolist()
        #list(int(i) for i in 6*pd.Series(fow)/pd.Series(fow_rr))
        w = [1] * len(fow)

        #### fetch last wkt stuff
        if (sub_df['legal_balls_bowled'].max()-max(fow_balls))>0:
            lwr = sub_df['runs_scored'].max()#-max(fow)
            lwb = sub_df['legal_balls_bowled'].max()#-max(fow_balls)

            fow.append(lwr)
            fow_balls.append(lwb)
            w.append(0)

        #fow_list.append(fow)
        #fow_balls_.append(fow_balls)
        #wkts.
        #print(m, fow, fow_balls, w)

        partnership_df.loc[i,'team'] = team
        partnership_df.loc[i,'Match'] = m
        partnership_df.iloc[i, range(2,len(fow)+2)] = fow
        partnership_df.iloc[i, range(12,len(fow_balls)+12)] = fow_balls
        partnership_df.iloc[i, range(22,len(w)+22)] = w


    
    
    

    for i in range(11, 2, -1):
        a = partnership_df.columns[i-1]
        b = partnership_df.columns[i]
        #print(a,b)
        partnership_df[b] = \
            partnership_df.apply(lambda row: safe_subtract(row[a], row[b]), axis=1)

    for i in range(21, 12, -1):
        a = partnership_df.columns[i-1]
        b = partnership_df.columns[i]
        #print(a,b)
        partnership_df[b] = \
            partnership_df.apply(lambda row: safe_subtract(row[a], row[b]), axis=1)




    wkt_list = ['1st','2nd','3rd',4,5,6,7,8,9,10]
    runs_ = []
    balls_ = []
    outs_ = []

    for w in wkt_list:
        collist = partnership_df.columns[partnership_df.columns.str.startswith(str(w))]
        #print(collist)
        sub_ = partnership_df[collist]
        run_col = collist[0]
        ball_col = collist[1]
        out_col = collist[2]
        runs = sub_[run_col].dropna().values
        balls = sub_[ball_col].dropna().values
        out = sub_[out_col].dropna().values
        #print(runs,balls, out)
        runs_.append(runs)
        balls_.append(balls)
        outs_.append(out)

    avg_runs = []
    avg_balls = []
    sum_runs = []
    outs = []

    for i in range(1,11):
        avg_runs.append(np.mean(runs_[i-1]))
        avg_balls.append(np.mean(balls_[i-1]))
        sum_runs.append(np.sum(runs_[i-1]))
        outs.append(np.sum(outs_[i-1]))

    p_df = pd.DataFrame([avg_runs, avg_balls,sum_runs, outs]).round(2)
    p_df = p_df.append(p_df.iloc[2,:]/p_df.iloc[3,:], ignore_index=True)

    p_df = p_df.T

    p_df['wkt_info'] = p_df.apply(
            lambda x: f"{x[0]}\n SR={np.round(100 * x[0] / x[1], 2)}, Avg={np.round(x[4],2)}",
            axis=1
        )

    #p_df = p_df['wkt_info']



    #p_df_mod = pd.DataFrame()
    print(team)

    #p_df_mod['wkt_info'] = p_df['wkt_info']
    
    p_df = p_df.T
    p_df.columns = ['1st Wkt','2nd Wkt','3rd Wkt','4th Wkt','5th Wkt',
                                          '6th Wkt','7th Wkt','8th Wkt','9th Wkt','10th Wkt']
    p_df['team'] = team

    p_df_all = pd.concat([p_df_all,p_df], axis=0, ignore_index=True)

CSK
DC
GT
LSG
KKR
MI
PBKS
RCB
RR
SRH


In [712]:
p_df_all = p_df_all.iloc[6*np.arange(10)+5, :]

In [714]:
p_df_all.to_excel('/Users/roumyadas/Desktop/IPL_Simulation/Stats/partnership_info.xlsx', index=None)

## pitch-wise bowling stats

In [487]:
#bowl_list = 'Natarajan|MP Yadav|Farooqi|AR Patel'
bowl_list = 'Bumrah'

#[df_sim.phase=='Death']\
pitch_ball = df_sim[df_sim.bowler.str.contains(bowl_list)]\
.groupby(['bowler','pitch','phase'])[['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

                        
                
pitch_ball['eco'] = 6*pitch_ball['runs_conceeded']/pitch_ball['islegal']
pitch_ball['SR'] = pitch_ball['islegal']/pitch_ball['isBowlerWicket']

pitch_ball = pitch_ball.rename(columns = {'islegal':'balls','runs_conceeded':'runs_conceded',
                                          'isBowlerWicket':'wkts'})

In [488]:
pitch_ball.round(2)

,bowler,pitch,phase,runs_conceded,balls,wkts,eco,SR
0,JJ Bumrah,neutralish,Death,28,18,4,9.33,4.50
1,JJ Bumrah,neutralish,Middle,32,18,0,10.67,inf
2,JJ Bumrah,neutralish,PP,25,12,0,12.50,inf
3,JJ Bumrah,pace,Death,142,82,3,10.39,27.33
4,JJ Bumrah,pace,Middle,221,180,15,7.37,12.00
5,JJ Bumrah,pace,PP,208,132,6,9.45,22.00
6,JJ Bumrah,spin,Death,75,48,0,9.38,inf
7,JJ Bumrah,spin,Middle,109,90,2,7.27,45.00
8,JJ Bumrah,spin,PP,123,72,4,10.25,18.00


## overall phase-bowling stats of bowlers

In [29]:
## phase-wise bowling stats

bowl_list = 'Mishra|Mujeeb'

phase_ball = df_sim[df_sim.bowler.str.contains(bowl_list)].groupby(['bowler','phase'])\
                        [['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

phase_ball['eco'] = 6*phase_ball['runs_conceeded']/phase_ball['islegal']
phase_ball['SR'] = phase_ball['islegal']/phase_ball['isBowlerWicket']

phase_ball = phase_ball.rename(columns = {'islegal':'balls','runs_conceeded':'runs_conceded',
                                          'isBowlerWicket':'wkts'})

In [30]:
phase_ball.round(2)

,bowler,phase,runs_conceded,balls,wkts,eco,SR
0,Mujeeb Ur Rahman,Death,86,60,5,8.60,12.0
1,Mujeeb Ur Rahman,Middle,18,12,0,9.00,inf
2,Mujeeb Ur Rahman,PP,110,72,3,9.17,24.0


In [389]:
df_sim.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'wicket_type', 'player_dismissed', 'legal_balls_bowled', 'wicket_event',
       'fielder', 'legality', 'bowler_wicket', 'islegal', 'isDotforBatter',
       'isOne', 'isTwo', 'isThree', 'isStrikeRotation', 'isFour', 'isSix',
       'isBowlerWicket', 'is_striker_Out', 'is_nonstriker_Out', 'isWicket',
       'total_runs', 'runs_scored', 'wickets_down', 'run_rate', 'last_fow',
       'partnership_runs', 'isno', 'is_faced_by_batter', 'last_fow_run_rate',
       'fours', 'sixes', 'striker_batting_position',
       'non_striker_batting_position', 'runs_conceeded', 'isDotforbowler',
       'legal_balls_remaining', 'target', 'runs_remaining', 'reqd_run_rate',
       'over', 'phase', 'before_ball_partnership', 'after_ball_partnership',
       'before_balls', 'after_balls', '

## inning-wise bowler stats

In [390]:
inn_ball = df_sim.groupby(['bowler','innings'])[['runs_conceeded','islegal','isBowlerWicket',
                                                'isDotforbowler']].sum().reset_index()

inn_ball.columns= ['bowler','innings','runs_conceded','balls','wkts','dots']

inn_ball['Eco'] = 6*inn_ball['runs_conceded']/inn_ball['balls']
inn_ball['SR'] = inn_ball['balls']/inn_ball['wkts']

inn_ball['runs_conceded'] = inn_ball['runs_conceded'].fillna(0).astype(int)
inn_ball['balls'] = inn_ball['balls'].fillna(0).astype(int)
inn_ball['wkts'] = inn_ball['wkts'].fillna(0).astype(int)
inn_ball = inn_ball.round(2)


In [391]:
#pat_ = 'KL|Livi|Markram|Lomror|VR Iyer|Duckett|Simran'
pat_ = 'Bumrah|Boult|HH Pandya|Abhishek|Theekshana'

inn_ball[(inn_ball['bowler'].str.contains(pat_))]\
                    .sort_values('bowler',ascending=False)

,bowler,innings,runs_conceded,balls,wkts,dots,Eco,SR
244,TA Boult,1,438,306,10,107,8.59,30.60
245,TA Boult,2,416,286,11,94,8.73,26.00
124,M Theekshana,1,226,157,5,36,8.64,31.40
125,M Theekshana,2,308,198,10,66,9.33,19.80
91,JJ Bumrah,1,527,334,16,80,9.47,20.88
92,JJ Bumrah,2,436,318,18,117,8.23,17.67
63,HH Pandya,1,504,314,19,87,9.63,16.53
64,HH Pandya,2,374,264,13,102,8.50,20.31
19,Abhishek Sharma,1,102,64,5,21,9.56,12.80
20,Abhishek Sharma,2,167,108,6,29,9.28,18.00


## inning-wise batter stats

In [121]:
inn_bat = df_sim.groupby(['striker','innings'])[['runs_off_bat','is_faced_by_batter']].sum().reset_index()
out_df = df_sim[df_sim.isWicket==1].groupby(['player_dismissed','innings'])['isWicket'].sum().reset_index()

out_df.columns= ['striker','innings','outs']
inn_bat = inn_bat.merge(out_df, on=['striker','innings'], how='outer')

inn_bat = inn_bat.rename(columns = {'runs_off_bat':'runs',
                                        'is_faced_by_batter':'balls'})

inn_bat['avg'] = inn_bat['runs']/inn_bat['outs']
inn_bat['SR'] = 100*inn_bat['runs']/inn_bat['balls']

inn_bat['runs'] = inn_bat['runs'].fillna(0).astype(int)
inn_bat['balls'] = inn_bat['balls'].fillna(0).astype(int)
inn_bat['outs'] = inn_bat['outs'].fillna(0).astype(int)
inn_bat = inn_bat.round(2)


In [387]:
#pat_ = 'KL|Livi|Markram|Lomror|VR Iyer|Duckett|Simran'
pat_ = 'McMullen|Raza'

inn_bat[(inn_bat['striker'].str.contains(pat_))]\
                    .sort_values('striker',ascending=False)

,striker,innings,runs,balls,outs,avg,SR
333,Sikandar Raza,1,198,151,4,49.50,131.13
334,Sikandar Raza,2,193,124,7,27.57,155.65
51,BJ McMullen,1,60,46,2,30.00,130.43
52,BJ McMullen,2,90,65,2,45.00,138.46


## analyzing the exit point stats for each batter

In [ ]:
def exit_pt_stats(batter, team):
    import warnings
    warnings.filterwarnings('ignore')
    import numpy as np

    # --- phase-wise stats
    phase_wise = df_sim[df_sim.striker == batter]\
                    .groupby(['match_id', 'phase'])[['runs_off_bat', 'is_faced_by_batter']]\
                    .sum().reset_index()

    # Create the performance descriptions
    phase_wise['PP_perf'] = phase_wise.apply(
        lambda x: f"{x['runs_off_bat']} off {x['is_faced_by_batter']}" if x['phase'] == 'PP' else np.nan,
        axis=1
    )
    phase_wise['Middle_perf'] = phase_wise.apply(
        lambda x: f"{x['runs_off_bat']} off {x['is_faced_by_batter']}" if x['phase'] == 'Middle' else np.nan,
        axis=1
    )
    phase_wise['Death_perf'] = phase_wise.apply(
        lambda x: f"{x['runs_off_bat']} off {x['is_faced_by_batter']}" if x['phase'] == 'Death' else np.nan,
        axis=1
    )

    # Now pivot it properly: one row per match_id
    phase_wise = phase_wise.groupby('match_id')[['PP_perf', 'Middle_perf', 'Death_perf']].first().reset_index()

    # --- match-wise stats
    match_wise = df_sim[df_sim.striker == batter]\
                    .groupby(['match_id', 'innings'])[['runs_off_bat', 'is_faced_by_batter']]\
                    .sum().reset_index()
    match_wise.columns = ['match_id', 'innings', 'runs', 'balls']
    match_wise['SR'] = (100 * match_wise['runs'] / match_wise['balls']).round(2)

    # --- final match score
    final_score = df_sim.groupby(['match_id', 'innings'])[['total_runs', 'islegal', 'isWicket']]\
                    .sum().reset_index()
    final_score.columns = ['match_id', 'innings', 'final_score', 'total_balls', 'wkts']

    # --- exit points
    exit_pts = df_sim[df_sim.player_dismissed == batter].groupby('match_id').tail(1).reset_index(drop=True)
    exit_pts['exit_pt_score'] = exit_pts.apply(
        lambda x: f"{x['runs_scored']}-{x['wickets_down']}({x['legal_balls_bowled']//6}.{x['legal_balls_bowled']%6})",
        axis=1
    )
    exit_pts = exit_pts[['match_id', 'exit_pt_score']]

    # --- merge
    match_exit = match_wise.merge(final_score.merge(exit_pts, on='match_id', how='left'), 
                                  on=['match_id', 'innings'], how='inner')

    # --- win/loss
    win_ = []
    for m in match_exit['match_id'].unique():
        t_runs = df_sim[(df_sim['match_id'] == m) & (df_sim['batting_team'] == team)]['runs_scored'].max()
        other_runs = df_sim[(df_sim['match_id'] == m) & (df_sim['batting_team'] != team)]['runs_scored'].max()
        win = 1 if t_runs > other_runs else 0
        win_.append(win)
    match_exit['win'] = win_

    # --- final merge
    match_exit = match_exit.merge(phase_wise, on='match_id', how='left')
    match_exit['batter'] = batter
    match_exit['team'] = team

    #print(f"{batter}, from {team}")
    return match_exit


In [ ]:
df_sim[df_sim['batting_team']=='CSK']['striker'].unique()

In [ ]:
batter_stats.columns

In [ ]:
batter_stats = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/batter_stats.csv')

batter_stats_mod = batter_stats[batter_stats.runs>=100]
exit_pt_df = pd.DataFrame()
for batter in batter_stats_mod['striker'].unique():
    batter = batter
    team = batter_stats_mod[batter_stats_mod.striker==batter]['batting_team'].unique()[0]
    exit_pt_df = exit_pt_df.append(exit_pt_stats(batter,team))

In [ ]:
exit_pt_df.head(10)

## phase-wise stats for each bowler

In [26]:
def phase_bowl_stats(bowler, team):
    import warnings
    warnings.filterwarnings('ignore')
    import numpy as np

    # --- phase-wise stats
    phase_wise = df_sim[df_sim.bowler == bowler]\
                    .groupby(['match_id', 'phase'])[['runs_conceeded', 'islegal','isBowlerWicket']]\
                    .sum().reset_index()

    # Create the performance descriptions
    phase_wise['PP_perf'] = phase_wise.apply(
        lambda x: f"{x['isBowlerWicket']}-{x['runs_conceeded']}({x['islegal']})" if x['phase'] == 'PP' else np.nan,
        axis=1
    )
    phase_wise['Middle_perf'] = phase_wise.apply(
        lambda x: f"{x['isBowlerWicket']}-{x['runs_conceeded']}({x['islegal']})" if x['phase'] == 'Middle' \
        else np.nan,
        axis=1
    )
    phase_wise['Death_perf'] = phase_wise.apply(
        lambda x: f"{x['isBowlerWicket']}-{x['runs_conceeded']}({x['islegal']})" if x['phase'] == 'Death' \
        else np.nan,
        axis=1
    )

    # Now pivot it properly: one row per match_id
    phase_wise = phase_wise.groupby('match_id')[['PP_perf', 'Middle_perf', 'Death_perf']].first().reset_index()

    # --- match-wise stats
    match_wise = df_sim[df_sim.bowler == bowler]\
                    .groupby(['match_id', 'innings'])[['runs_conceeded', 'islegal','isBowlerWicket']]\
                    .sum().reset_index()
    match_wise.columns = ['match_id', 'innings', 'runs', 'balls','wkts']
    match_wise['Eco'] = (6 * match_wise['runs'] / match_wise['balls']).round(2)

    # --- final match score
    final_score = df_sim.groupby(['match_id', 'innings'])[['total_runs', 'islegal', 'isWicket']]\
                    .sum().reset_index()
    final_score.columns = ['match_id', 'innings', 'final_score', 'total_balls', 'wkts_lost']

    # --- merge
    match_exit = match_wise.merge(final_score, 
                                  on=['match_id', 'innings'], how='inner')

    # --- win/loss
    win_ = []
    for m in match_exit['match_id'].unique():
        t_runs = df_sim[(df_sim['match_id'] == m) & (df_sim['batting_team'] == team)]['runs_scored'].max()
        other_runs = df_sim[(df_sim['match_id'] == m) & (df_sim['batting_team'] != team)]['runs_scored'].max()
        win = 1 if t_runs > other_runs else 0
        win_.append(win)
    match_exit['win'] = win_

    # --- final merge
    match_exit = match_exit.merge(phase_wise, on='match_id', how='left')
    match_exit['bowler'] = bowler
    match_exit['team'] = team

    #print(f"{bowler}, from {team}")
    return match_exit


bowler_stats = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/bowler_stats.csv')

bowler_stats_mod = bowler_stats[bowler_stats.balls>=24]
phase_bowl_stat = pd.DataFrame()
for bowler in bowler_stats_mod['bowler'].unique():
    bowler = bowler
    team = bowler_stats_mod[bowler_stats_mod.bowler==bowler]['bowling_team'].unique()[0]
    phase_bowl_stat = phase_bowl_stat.append(phase_bowl_stats(bowler,team))

In [27]:
phase_bowl_stat[(phase_bowl_stat['team']=='RR')&(phase_bowl_stat['innings']==2)]

NameError: name 'phase_bowl_stat' is not defined

In [ ]:
# livi = death
# kuldeep = pp, middle
# bethell = middle
# porel = pp
# naveen = death
# badoni = middle
# venky = middle

## fastest to get to 50, 100

In [ ]:
batters = df_sim.striker.unique().tolist()
dfs = []

for batter in batters:
    sub_df = df_sim[df_sim.striker == batter].copy()
    sub_df[['runs', 'balls']] = sub_df.groupby(['match_id','batting_team','bowling_team'])[['runs_off_bat', 'is_faced_by_batter']].cumsum()

    df = sub_df[sub_df.runs >= 100].groupby(['match_id','batting_team','bowling_team']).first().reset_index()  # take the first 50+ entry per match
    df['striker'] = batter  # add batter info
    dfs.append(df[['striker', 'match_id','batting_team', 'balls', 'bowling_team']])  # keep only needed columns

final_df = pd.concat(dfs, ignore_index=True)


final_df.sort_values('balls')

## fastest to get to 3wickets, 5wickets

In [ ]:
bowlers = df_sim.bowler.unique().tolist()
dfs = []

for bowler in bowlers:
    sub_df = df_sim[df_sim.bowler == bowler].copy()
    sub_df[['balls', 'wkts']] = sub_df.groupby(['match_id','bowling_team','batting_team'])[['islegal','isBowlerWicket']].cumsum()

    df = sub_df[sub_df.wkts >= 5].groupby(['match_id','bowling_team','batting_team']).first().reset_index()  
    df['bowler'] = bowler  # add bowler info
    dfs.append(df[['bowler', 'match_id', 'bowling_team','batting_team', 'balls']])  # keep only needed columns

final_df = pd.concat(dfs, ignore_index=True)


final_df.sort_values('balls')

## fastest, and slowest knocks 20+ balls

In [ ]:
batters = df_sim.striker.unique().tolist()
dfs = []

for batter in batters:
    sub_df = df_sim[df_sim.striker == batter].copy()
    sub_df = sub_df.groupby(['match_id','batting_team','bowling_team'])[['runs_off_bat', 'is_faced_by_batter']].sum().reset_index()

    sub_df = sub_df[sub_df.is_faced_by_batter >= 10] #####
    sub_df['striker'] = batter  # add batter info
    dfs.append(sub_df[['striker', 'match_id','batting_team', 'runs_off_bat', 'is_faced_by_batter', 'bowling_team']])  # keep only needed columns

final_df = pd.concat(dfs, ignore_index=True)
final_df['SR'] = 100*final_df['runs_off_bat']/final_df['is_faced_by_batter']



final_df.sort_values('SR', ascending=False)

## checking who excels in first 2 wkt vs 3-5 wkt partnerships

In [ ]:
wkts = 3

p_df = df_sim[(df_sim.isWicket==1)&(df_sim.wickets_down<=wkts)].groupby('batting_team')\
            [['partnership_runs','isWicket','after_balls']].sum().reset_index()
p_df['avg'] = p_df['partnership_runs']/p_df['isWicket']
p_df['SR'] = 100*p_df['partnership_runs']/p_df['after_balls']

p_df

In [ ]:
wkts = [4,7]

p_df = df_sim[(df_sim.isWicket==1)&(df_sim.wickets_down<=wkts[1])&(df_sim.wickets_down<=wkts[0])]\
                    .groupby('batting_team')\
            [['partnership_runs','isWicket','after_balls']].sum().reset_index()
p_df['avg'] = p_df['partnership_runs']/p_df['isWicket']
p_df['SR'] = 100*p_df['partnership_runs']/p_df['after_balls']

p_df

## checking over-reliance on a batter

In [ ]:
batter = 'BM Duckett'
team = 'SRH'

print(6*df_sim[(df_sim['striker']==batter)|(df_sim['non_striker']==batter)]['total_runs'].sum()/\
        df_sim[(df_sim['striker']==batter)|(df_sim['non_striker']==batter)]['islegal'].sum())

print(6*df_sim[(df_sim.wickets_down<=7)&(df_sim.batting_team==team)&(df_sim['striker']!=batter)&(df_sim['non_striker']!=batter)]['total_runs'].sum()/\
    df_sim[(df_sim.wickets_down<=7)&(df_sim.batting_team==team)&(df_sim['striker']!=batter)&(df_sim['non_striker']!=batter)]['islegal'].sum())






In [ ]:
df_sim[df_sim.striker=='Tilak Varma'].groupby(['match_id','striker'])\
            [['runs_off_bat','is_faced_by_batter']].sum().reset_index()

## batting position wise team performances

In [ ]:
pos_tm = df_sim.groupby(['striker_batting_position','batting_team'])\
                [['runs_off_bat','is_faced_by_batter']].sum().reset_index()

pos_tm.columns = ['order','team','runs','balls']
df_sim['out_bat_position'] = df_sim[['isWicket','player_dismissed','striker','non_striker',
                                'striker_batting_position','non_striker_batting_position']]\
                    .apply(lambda x: x[4] if x[1]==x[2] else (x[5] if x[1]==x[3] else None), axis=1)

tm_pos_outs = df_sim.groupby(['batting_team','out_bat_position'])['isWicket'].sum().reset_index()
tm_pos_outs.columns = ['team','order','outs']
pos_tm  = pos_tm.merge(tm_pos_outs, on=['team','order'], how='outer')

pos_tm['avg'] = pos_tm['runs']/pos_tm['outs']
pos_tm['SR'] = 100*pos_tm['runs']/pos_tm['balls']
pos_tm['outs'] = pos_tm['outs'].fillna(0)
pos_tm['outs'] = pos_tm['outs'].astype(int)
pos_tm = pos_tm.round(2)

In [ ]:
pos_tm[pos_tm['team']=='DC']

In [ ]:
str_pos = df_sim.groupby(['striker_batting_position','striker'])\
                [['runs_off_bat','is_faced_by_batter']].sum().reset_index()
str_pos[str_pos['striker']=='N Pooran']

## phase-wise batter stats

In [41]:
phase_bat = df_sim.groupby(['striker','phase'])[['runs_off_bat','is_faced_by_batter']].sum().reset_index()
out_df = df_sim[df_sim.isWicket==1].groupby(['player_dismissed','phase'])['isWicket'].sum().reset_index()

out_df.columns= ['striker','phase','outs']
phase_bat = phase_bat.merge(out_df, on=['striker','phase'], how='outer')

phase_bat = phase_bat.rename(columns = {'runs_off_bat':'runs',
                                        'is_faced_by_batter':'balls'})

phase_bat['avg'] = phase_bat['runs']/phase_bat['outs']
phase_bat['SR'] = 100*phase_bat['runs']/phase_bat['balls']

phase_bat['runs'] = phase_bat['runs'].fillna(0).astype(int)
phase_bat['balls'] = phase_bat['balls'].fillna(0).astype(int)
phase_bat['outs'] = phase_bat['outs'].fillna(0).astype(int)
phase_bat = phase_bat.round(2)


In [51]:
#pat_ = 'Simran|Stubbs|VR Iyer|KL Rahul|Lomror|Livi|Markram|Priyansh Arya'
pat_ = 'Stokes'

phase_bat[(phase_bat['striker'].str.contains(pat_))]\
                    .sort_values('striker',ascending=False)

,striker,phase,runs,balls,outs,avg,SR
61,BA Stokes,Death,36,23,2,18.00,156.52
62,BA Stokes,Middle,212,141,7,30.29,150.35
63,BA Stokes,PP,245,174,13,18.85,140.80


## phase-wise bowling stats

In [106]:
phase_ball = df_sim.groupby(['bowler','bowling_team','phase'])\
                        [['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

phase_ball['eco'] = 6*phase_ball['runs_conceeded']/phase_ball['islegal']
phase_ball['SR'] = phase_ball['islegal']/phase_ball['isBowlerWicket']

In [107]:
phase_ball[(phase_ball.bowling_team=='SRH')\
           &(phase_ball.phase=='Death')]\
                            .sort_values(['phase','eco'], ascending=True)

,bowler,bowling_team,phase,runs_conceeded,islegal,isBowlerWicket,eco,SR
20,AJ Hosein,SRH,Death,10,12,1,5.000000,12.000000
14,A Zampa,SRH,Death,62,62,7,6.000000,8.857143
302,Mukesh Kumar,SRH,Death,12,12,0,6.000000,inf
159,JG Bethell,SRH,Death,19,18,2,6.333333,9.000000
134,IC Porel,SRH,Death,7,6,0,7.000000,inf
394,SH Johnson,SRH,Death,81,62,7,7.838710,8.857143
68,C Sakariya,SRH,Death,70,50,2,8.400000,25.000000
452,Tanush Kotian,SRH,Death,71,48,7,8.875000,6.857143
202,LH Ferguson,SRH,Death,9,6,0,9.000000,inf
23,AK Markram,SRH,Death,157,102,10,9.235294,10.200000


In [ ]:
phase_ball = df_sim.groupby(['bowler','bowling_team','innings',
                             'phase'])\
                        [['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

phase_ball['eco'] = 6*phase_ball['runs_conceeded']/phase_ball['islegal']
phase_ball['SR'] = phase_ball['islegal']/phase_ball['isBowlerWicket']

In [ ]:
phase_ball[(phase_ball.bowling_team=='SRH')&(phase_ball.innings==1)\
           &(phase_ball.phase=='Middle')]\
                            .sort_values(['phase','eco'], ascending=True)

In [ ]:
phase_ball_tm = df_sim.groupby(['bowling_team','phase'])\
                        [['runs_conceeded','islegal','isWicket']].sum().reset_index()

phase_ball_tm['eco'] = 6*phase_ball_tm['runs_conceeded']/phase_ball_tm['islegal']
phase_ball_tm['SR'] = phase_ball_tm['islegal']/phase_ball_tm['isWicket']

phase_ball_tm = phase_ball_tm.rename(columns={'isWicket':'wkts'})

In [ ]:
phase_ball_tm[phase_ball_tm['bowling_team']=="SRH"][['bowling_team','phase','wkts','eco','SR']].round(2)

In [ ]:
phase_bat_tm = df_sim.groupby(['batting_team','phase'])\
                        [['total_runs','islegal','isWicket']].sum().reset_index()

phase_bat_tm['avg'] = phase_bat_tm['total_runs']/phase_bat_tm['isWicket']
phase_bat_tm['SR'] = 100*phase_bat_tm['total_runs']/phase_bat_tm['islegal']

phase_bat_tm = phase_bat_tm.rename(columns={'isWicket':'outs'})


phase_bat_tm = phase_bat_tm[['batting_team','phase','outs','avg','SR']].round(2)
phase_bat_tm = phase_bat_tm.sort_values(by='batting_team')


df_sim['match_id'].unique()

In [ ]:
team = "SRH"
df_sim[(df_sim['batting_team']==team)|(df_sim['bowling_team']==team)]['match_id'].unique()

In [ ]:
#reading all STATS from one excel##
excel_filename = "/Users/roumyadas/Desktop/IPL_Simulation/Season_01/stats/STATS_S01.xlsx"

batter_stats = pd.read_excel(excel_filename, sheet_name='BAT')
bowler_stats = pd.read_excel(excel_filename, sheet_name='BOWL')

In [ ]:
batter_stats = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/batter_stats.csv')
bowler_stats = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/bowler_stats.csv')


In [ ]:
df_sim.shape

In [ ]:
df_sim.tail(5)

# Partnership setting

In [748]:
p_out_df = df_sim[df_sim.season==2][df_sim.isWicket==1].groupby('wickets_down')[['after_ball_partnership']].max().reset_index()

#p_df

df_sim[df_sim.season==2][df_sim.isWicket==1].merge(p_out_df, on=['wickets_down','after_ball_partnership'], how='inner')\
        [['match_id','bowling_team','batting_team','striker','non_striker','runs_scored','wickets_down','over',
'after_ball_partnership','after_balls']].sort_values(by='wickets_down')

,match_id,bowling_team,batting_team,striker,non_striker,runs_scored,wickets_down,over,after_ball_partnership,after_balls
2,S02M022,CSK,SRH,P Simran Singh,BM Duckett,142,1,16.5,142,101
3,S02M034,SRH,MI,PP Shaw,N Pooran,181,2,18.5,149,91
6,S02M040,KKR,CSK,F du Plessis,T Stubbs,200,3,19.3,190,109
8,S02M062,MI,RR,YBK Jaiswal,S Dube,182,4,17.2,135,63
4,S02M035,LSG,KKR,RA Bawa,R Powell,146,5,18.2,57,28
9,S02M066,LSG,CSK,F du Plessis,AR Patel,143,6,18.0,61,40
0,S02M009,GT,LSG,Abdul Samad,SP Narine,149,7,18.4,60,37
7,S02M047,LSG,MI,M Theekshana,Jay Gohil,175,8,19.2,34,24
1,S02M010,KKR,RR,S Dube,JR Hazlewood,135,9,17.1,19,7
5,S02M038,SRH,PBKS,M Markande,Shashank Singh,166,10,19.5,25,20


In [749]:
p_no_df = df_sim[df_sim.season==2][df_sim.isWicket!=1].groupby('wickets_down')[['after_ball_partnership']].max().reset_index()

#p_df

df_sim[df_sim.season==2][df_sim.isWicket!=1].merge(p_no_df, on=['wickets_down','after_ball_partnership'], how='inner')\
        [['match_id','bowling_team','batting_team','striker','non_striker','runs_scored','wickets_down','over',
'after_ball_partnership',\
          'after_balls']].sort_values(by='wickets_down')

,match_id,bowling_team,batting_team,striker,non_striker,runs_scored,wickets_down,over,after_ball_partnership,after_balls
8,S02M022,CSK,SRH,BM Duckett,P Simran Singh,142,0,16.4,142,100
9,S02M034,SRH,MI,PP Shaw,N Pooran,181,1,18.4,149,90
10,S02M040,KKR,CSK,F du Plessis,T Stubbs,200,2,19.2,190,108
13,S02M062,MI,RR,YBK Jaiswal,S Dube,182,3,17.1,135,62
12,S02M055,KKR,RCB,JM Sharma,RK Singh,188,4,20.0,77,44
14,S02M063,RCB,PBKS,Washington Sundar,RG Sharma,184,5,20.0,97,56
0,S02M009,GT,LSG,SP Narine,Abdul Samad,149,6,18.3,60,36
11,S02M047,LSG,MI,M Theekshana,Jay Gohil,175,7,19.1,34,23
1,S02M010,KKR,RR,S Dube,JR Hazlewood,135,8,16.5,19,5
2,S02M010,KKR,RR,JR Hazlewood,S Dube,135,8,17.0,19,6


In [ ]:
df_sim_copy = df_sim.copy()

df_sim_copy['over_mod'] = np.where(df_sim_copy['over_']<=6, '1-6', \
                          np.where(df_sim_copy['over_']<=10, '7-10', \
                          np.where(df_sim_copy['over_']<=15, '11-15', \
                          '16-20')))

In [ ]:
bat_pos = df_sim.groupby('striker_batting_position').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

bat_pos['SR'] = 100*bat_pos['runs']/bat_pos['balls']
bat_pos['Bat_avg'] = bat_pos['runs']/bat_pos['outs']
bat_pos['bpb'] = bat_pos['balls']/(bat_pos['fours']+bat_pos['sixes'])

bat_pos.sort_values('striker_batting_position',ascending=True)

In [ ]:
df_sim.columns

In [ ]:
bowl_over = df_sim_copy.groupby('over_mod').agg(
    runs = ('runs_conceeded','sum'),
    balls = ('islegal','sum'),
    outs = ('isBowlerWicket','sum'),
    dots = ('isDotforbowler','sum')
).reset_index()

bowl_over['SR'] = bowl_over['balls']/bowl_over['outs']
bowl_over['Economy'] = 6*bowl_over['runs']/bowl_over['balls']
bowl_over['avg'] = bowl_over['runs']/bowl_over['outs']
bowl_over['dot%'] = 100*bowl_over['dots']/bowl_over['balls']


bowl_over.sort_values('over_mod',ascending=True)

In [ ]:
df_sim[df_sim['wicket_event']=='Caught'].groupby('fielder')['wicket_type'].count().reset_index()\
                .sort_values(by='wicket_type',ascending=False)

In [398]:
df_sim.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'wicket_type', 'player_dismissed', 'legal_balls_bowled', 'wicket_event',
       'fielder', 'legality', 'bowler_wicket', 'islegal', 'isDotforBatter',
       'isOne', 'isTwo', 'isThree', 'isStrikeRotation', 'isFour', 'isSix',
       'isBowlerWicket', 'is_striker_Out', 'is_nonstriker_Out', 'isWicket',
       'total_runs', 'runs_scored', 'wickets_down', 'run_rate', 'last_fow',
       'partnership_runs', 'isno', 'is_faced_by_batter', 'last_fow_run_rate',
       'fours', 'sixes', 'striker_batting_position',
       'non_striker_batting_position', 'runs_conceeded', 'isDotforbowler',
       'legal_balls_remaining', 'target', 'runs_remaining', 'reqd_run_rate',
       'over', 'phase', 'before_ball_partnership', 'after_ball_partnership',
       'before_balls', 'after_balls', '

## inning,pitch-wise win count

In [448]:
team = "PBKS"

sub_df = df_sim[(df_sim.batting_team==team)|(df_sim.bowling_team==team)][df_sim.match_id.str.startswith('S')]

i_list = []
win_list = []
p_list = []

for m in sub_df.match_id.unique():
    bat_t = sub_df[sub_df.match_id==m][sub_df.batting_team==team]['runs_scored'].max()
    bat_o = sub_df[sub_df.match_id==m][sub_df.batting_team!=team]['runs_scored'].max()
    inn_t = sub_df[sub_df.match_id==m][sub_df.batting_team==team]['innings'].unique()[0]
    inn_o = sub_df[sub_df.match_id==m][sub_df.batting_team!=team]['innings'].unique()[0]
    pitch = sub_df[sub_df.match_id==m]['pitch'].unique()[0]
    
    if bat_t>bat_o:
        w = 1
    else:
        w = 0
    
    i_list.append(inn_t)
    win_list.append(w)
    p_list.append(pitch)

win_df = pd.DataFrame()
win_df['inning'] = i_list
win_df['pitch'] = p_list
win_df['win'] = win_list
win_df['team'] = team

In [449]:
g_cols = ['pitch','inning']

g_cols.append('team')

win_ = win_df.groupby(g_cols).agg(
    wins = ('win','sum'),
    matches = ('win','count')).reset_index()

win_['win%'] = 100*win_['wins']/win_['matches']

win_.round(2)

,pitch,inning,team,wins,matches,win%
0,neutralish,1,PBKS,0,1,0.0
1,pace,1,PBKS,0,2,0.0
2,pace,2,PBKS,0,3,0.0
3,spin,1,PBKS,2,4,50.0
4,spin,2,PBKS,3,4,75.0


## venue-wise win count

In [446]:
v_list = []
w_list = []
ha_list = []

for m_id in df_sim[df_sim.match_id.str.startswith('S')]['match_id'].unique()[:74]:
    sub_df = df_sim[df_sim.match_id.str.startswith('S')][df_sim['match_id']==m_id]
    
    v = sub_df['venue'].unique()[0]
    bat_1 = sub_df[sub_df.innings==1]['runs_scored'].max()
    bat_2 = sub_df[sub_df.innings==2]['runs_scored'].max()
    bat_1_t = sub_df[sub_df.innings==1]['batting_team'].unique()[0]
    bat_2_t = sub_df[sub_df.innings==2]['batting_team'].unique()[0]
    
    if bat_1>bat_2:
        w = bat_1_t
    else:
        w = bat_2_t
        
    if v==w:
        ha_list.append('h')
    else:
        ha_list.append('a')
    
    v_list.append(v)
    w_list.append(w)
    
win_df = pd.DataFrame()
win_df['venue'] = v_list
win_df['winner'] = w_list
win_df['home_away'] = ha_list
win_df['wins'] = 1

win_df = win_df.groupby(['home_away','winner'])['wins'].sum().reset_index()

In [447]:
win_df.sort_values(by=['home_away','wins'], ascending=False).reset_index(drop=True)

,home_away,winner,wins
0,h,CSK,6
1,h,SRH,6
2,h,DC,5
3,h,LSG,4
4,h,MI,4
5,h,PBKS,4
6,h,RR,4
7,h,KKR,3
8,h,GT,2
9,h,RCB,2


# POTT points

In [ ]:
batter = batter_stats
bowler = bowler_stats

In [ ]:
batter[batter.striker.str.contains('Livingstone')]

In [ ]:
bowler[bowler.bowler.str.contains('Livingstone')]

In [ ]:
bowler[bowler.balls>=90].sort_values('bpb', ascending=False).head(5)

In [ ]:
batter.sample(4)

In [ ]:
# runs * 0.5
# balls/dismissal * 0.5
# SR * 0.5
# fours * 0.25 + sixes * 0.5
# dot_% * (-2)
# 1/bpb * 50
# 

batter['points'] = batter['runs']*0.5 + \
            np.where(batter['balls_per_dismissal']<1000, batter['balls_per_dismissal']*0.5, batter['balls']*0.5)\
                   + batter['strike_rate']*0.5+\
                   batter['fours']*0.25 + batter['sixes']*0.5 + batter['dot_%']*(-0.6) + \
                   np.where(batter['bpb']>0, 50/batter['bpb'], 0)



In [ ]:
batter.sort_values('points', ascending=False).head(20)

In [ ]:
bowler.sample(5)

In [ ]:
# wickets * 12.5
# balls/dismissal = 1/SR * 500
# 1/economy * 100
# fours * (-0.25) + sixes * (-0.5)
# dot_% * 2
# bpb * 5
# 

bowler['points'] = bowler['wkts']*12.5 + \
            np.where(bowler['strike_rate']<1000, 500/bowler['strike_rate'], 0)\
                   + 100/bowler['economy']+\
                   bowler['fours']*(-0.25) + bowler['sixes']*(-0.5) + bowler['dot_%']*0.6 + \
                   np.where(bowler['bpb']>0, bowler['bpb']*5, bowler['balls'])



In [ ]:
bowler.sort_values('points', ascending=False).head(20)

In [ ]:
batter_mod = batter.copy()
bowler_mod = bowler.copy()
batter_mod.rename(columns={'striker':'player','points':'points_bat'},inplace=True)
bowler_mod.rename(columns={'bowler':'player','points':'points_ball'},inplace=True)

batter_mod = batter_mod[['player','points_bat']]
bowler_mod = bowler_mod[['player','points_ball']]

player_stats = batter_mod.merge(bowler_mod, on='player',how='outer')
player_stats.fillna(0, inplace=True)
player_stats['points'] = player_stats['points_bat']+player_stats['points_ball']

player_stats.sort_values('points',ascending=False).head(30)


## comment ::  Mitch Marsh comfortably POTT

In [ ]:
player_stats.sort_values('points',ascending=False).iloc[0:50,:]

In [ ]:
player_stats[(player_stats.points_bat>=150)&(player_stats.points_ball>=150)]\
            .sort_values('points',ascending=False).head(15)


## POTT, at less matches

In [ ]:
player_stats['matches_inv'] = 0 
for index,row in player_stats.iterrows():
    player = row['player']
    matches = df_sim[(df_sim.striker==player)|(df_sim.bowler==player)].match_id.nunique()
    player_stats.at[index, 'matches_inv'] = matches

In [ ]:
player_stats['pts/mt'] = player_stats['points']/player_stats['matches_inv']
#(player_stats.matches_inv>=5)&
player_stats[(player_stats.matches_inv<=10)].sort_values('pts/mt',ascending=False).head(20)

In [ ]:
bowler[bowler.bowler.str.contains('Dubey')]

In [ ]:
df_sim.columns

## picking best XI

In [303]:
#Batting position wise
#[df_sim.phase=='PP']

bat_pos = df_sim[df_sim['match_id'].str.startswith('S02')]\
                    .groupby(['striker_batting_position','striker']).agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum')
).reset_index()

bat_pos['SR'] = 100*bat_pos['runs']/bat_pos['balls']
bat_pos['Bat_avg'] = bat_pos['runs']/bat_pos['outs']
bat_pos['bpb'] = bat_pos['balls']/(bat_pos['fours']+bat_pos['sixes'])
bat_pos['dot%'] = 100*bat_pos['dots']/bat_pos['balls']


In [360]:
bat_open = df_sim[df_sim['match_id'].str.startswith('S02')]\
                [df_sim.striker_batting_position.isin([1,2])].groupby(['striker']).agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum')
).reset_index()

avg = 30
ball = 100
bat_open['SR'] = 100*bat_open['runs']/bat_open['balls']
bat_open['Bat_avg'] = bat_open['runs']/bat_open['outs']
bat_open['bpb'] = bat_open['balls']/(bat_open['fours']+bat_open['sixes'])
bat_open['dot%'] = 100*bat_open['dots']/bat_open['balls']


bat_open[bat_open.Bat_avg>=avg][bat_open.balls>=ball][bat_open.striker!='GJ Maxwell']\
                    .sort_values(['runs','SR','Bat_avg'],ascending=False).round(2).head(15)



,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot%
30,RD Rickelton,646,405,17,53,31,123,159.51,38.00,4.82,30.37
8,DP Conway,620,386,13,65,18,96,160.62,47.69,4.65,24.87
25,PD Salt,536,324,13,51,23,92,165.43,41.23,4.38,28.40
29,RD Gaikwad,535,341,11,48,19,83,156.89,48.64,5.09,24.34
37,Shubman Gill,512,328,10,45,20,93,156.10,51.20,5.05,28.35
41,V Kohli,509,311,10,43,21,72,163.67,50.90,4.86,23.15
24,P Simran Singh,502,316,14,42,21,84,158.86,35.86,5.02,26.58
15,JC Buttler,483,299,14,51,18,87,161.54,34.50,4.33,29.10
36,SV Samson,471,299,10,40,18,77,157.53,47.10,5.16,25.75
38,TM Head,457,314,12,50,12,90,145.54,38.08,5.06,28.66


In [361]:
## pick 3,4

bat_3_4_5 = df_sim[df_sim['match_id'].str.startswith('S02')]\
                [df_sim.striker_batting_position.isin([3,4,5])].groupby(['striker']).agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum')
).reset_index()

avg = 30
ball = 100
bat_3_4_5['SR'] = 100*bat_3_4_5['runs']/bat_3_4_5['balls']
bat_3_4_5['Bat_avg'] = bat_3_4_5['runs']/bat_3_4_5['outs']
bat_3_4_5['bpb'] = bat_3_4_5['balls']/(bat_3_4_5['fours']+bat_3_4_5['sixes'])
bat_3_4_5['dot%'] = 100*bat_3_4_5['dots']/bat_3_4_5['balls']


bat_3_4_5[bat_3_4_5.Bat_avg>=avg][bat_3_4_5.balls>=ball]\
                    .sort_values(['runs','SR','Bat_avg'],ascending=False).round(2).head(15)


,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot%
62,SS Iyer,588,369,12,48,23,81,159.35,49.00,5.20,21.95
59,SA Yadav,586,338,10,51,22,53,173.37,58.60,4.63,15.68
40,N Pooran,558,332,7,44,25,67,168.07,79.71,4.81,20.18
31,KL Rahul,538,339,3,45,18,66,158.70,179.33,5.38,19.47
1,AM Rahane,537,368,11,47,19,107,145.92,48.82,5.58,29.08
58,S Dube,498,306,9,35,27,82,162.75,55.33,4.94,26.80
56,RM Patidar,464,318,14,38,19,89,145.91,33.14,5.58,27.99
78,YBK Jaiswal,437,233,6,35,22,46,187.55,72.83,4.09,19.74
43,Naman Dhir,434,263,7,32,14,38,165.02,62.00,5.72,14.45
5,Abishek Porel,432,235,8,34,25,57,183.83,54.00,3.98,24.26


In [365]:
## pick 5

bat_4_5_6 = df_sim[df_sim['match_id'].str.startswith('S02')]\
            [df_sim.striker_batting_position.isin([4,5,6])].groupby(['striker']).agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum')
).reset_index()

avg = 30
ball = 80
bat_4_5_6['SR'] = 100*bat_4_5_6['runs']/bat_4_5_6['balls']
bat_4_5_6['Bat_avg'] = bat_4_5_6['runs']/bat_4_5_6['outs']
bat_4_5_6['bpb'] = bat_4_5_6['balls']/(bat_4_5_6['fours']+bat_4_5_6['sixes'])
bat_4_5_6['dot%'] = 100*bat_4_5_6['dots']/bat_4_5_6['balls']


bat_4_5_6[bat_4_5_6.Bat_avg>=avg][bat_4_5_6.balls>=ball][bat_4_5_6.striker!='N Pooran']\
                    .sort_values(['runs','SR','Bat_avg'],ascending=False).round(2).head(15)


,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot%
59,S Dube,534,329,10,39,28,87,162.31,53.40,4.91,26.44
6,Abishek Porel,464,255,11,38,26,63,181.96,42.18,3.98,24.71
77,YBK Jaiswal,437,233,6,35,22,46,187.55,72.83,4.09,19.74
76,VR Iyer,395,260,7,39,15,71,151.92,56.43,4.81,27.31
70,Smaran Ravichandran,388,228,9,27,15,52,170.18,43.11,5.43,22.81
71,T Stubbs,356,224,6,26,16,44,158.93,59.33,5.33,19.64
32,KL Rahul,309,186,2,27,9,31,166.13,154.50,5.17,16.67
55,RK Singh,307,196,7,23,12,41,156.63,43.86,5.60,20.92
50,R Powell,304,178,9,22,18,48,170.79,33.78,4.45,26.97
24,H Klaasen,282,183,5,20,12,37,154.10,56.40,5.72,20.22


In [367]:
## pick 6

bat_5_6_7 = df_sim[df_sim['match_id'].str.startswith('S02')]\
                [df_sim.striker_batting_position.isin([5,6,7])].groupby(['striker']).agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum')
).reset_index()

avg = 30
ball = 80
bat_5_6_7['SR'] = 100*bat_5_6_7['runs']/bat_5_6_7['balls']
bat_5_6_7['Bat_avg'] = bat_5_6_7['runs']/bat_5_6_7['outs']
bat_5_6_7['bpb'] = bat_5_6_7['balls']/(bat_5_6_7['fours']+bat_5_6_7['sixes'])
bat_5_6_7['dot%'] = 100*bat_5_6_7['dots']/bat_5_6_7['balls']


bat_5_6_7[bat_5_6_7.Bat_avg>=avg][bat_5_6_7.balls>=ball][bat_5_6_7.striker!='YBK Jaiswal']\
                    .sort_values(['runs','SR','Bat_avg'],ascending=False).round(2).head(15)


,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot%
53,R Powell,334,193,9,24,21,54,173.06,37.11,4.29,27.98
6,Abishek Porel,306,165,9,28,17,44,185.45,34.00,3.67,26.67
23,H Klaasen,246,162,4,18,10,34,151.85,61.50,5.79,20.99
31,KA Pollard,217,142,5,15,12,42,152.82,43.40,5.26,29.58
0,AD Russell,211,116,3,17,12,29,181.90,70.33,4.00,25.00
58,RK Singh,205,131,4,18,6,24,156.49,51.25,5.46,18.32
60,RR Rossouw,199,120,6,16,8,23,165.83,33.17,5.00,19.17
29,JM Sharma,180,98,4,10,14,24,183.67,45.00,4.08,24.49


In [382]:
bat_5_6_7[bat_5_6_7.Bat_avg<avg][bat_5_6_7.balls<ball]\
                    .sort_values(['runs','SR','Bat_avg'],ascending=False).head(15)


,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot%
24,HH Pandya,110,79,8,4,8,28,139.240506,13.750000,6.583333,35.443038
55,RA Bawa,88,50,5,6,7,20,176.000000,17.600000,3.846154,40.000000
65,SM Curran,74,72,5,5,2,26,102.777778,14.800000,10.285714,36.111111
79,Urvil Patel,67,32,3,6,3,5,209.375000,22.333333,3.555556,15.625000
19,DJ Hooda,61,34,4,3,5,11,179.411765,15.250000,4.250000,32.352941
17,DA Miller,56,51,2,2,3,19,109.803922,28.000000,10.200000,37.254902
34,LS Livingstone,46,35,2,2,3,13,131.428571,23.000000,7.000000,37.142857
40,MP Stoinis,43,26,2,1,3,6,165.384615,21.500000,6.500000,23.076923
7,Anmolpreet Singh,43,31,2,6,0,7,138.709677,21.500000,5.166667,22.580645
71,Shakib Al Hasan,40,33,2,2,3,16,121.212121,20.000000,6.600000,48.484848


In [364]:
names_ex = 'Ghazanfar|AR Patel|Natarajan|Sundar|Yadav|Varun'
bowler[~bowler.bowler.str.contains(names_ex)].head(10)#[bowler.bowler=='M Pathirana']

AttributeError: 'str' object has no attribute 'bowler'

In [ ]:
##powerplay & Middle overs

pp_mid_bowl = df_sim[df_sim.over_<=15].groupby('bowler').agg(
    team = ('bowling_team','unique'),
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    wkts = ('isBowlerWicket','sum'),
    dots = ('isDotforbowler','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

pp_mid_bowl['Eco'] = 6*pp_mid_bowl['runs']/pp_mid_bowl['balls']
pp_mid_bowl['bdry'] = pp_mid_bowl['fours']+pp_mid_bowl['sixes']
pp_mid_bowl['SR'] = pp_mid_bowl['balls']/pp_mid_bowl['wkts']

eco = 10
sr = 12
pp_mid_bowl[pp_mid_bowl.balls>=60][pp_mid_bowl.SR<=sr][pp_mid_bowl.Eco<eco].sort_values('Eco',ascending=True).head(15)


df_sim.sample()

In [ ]:
runs_table = pd.pivot_table(data=df_sim, values='total_runs', index="match_id", columns='batting_team', 
                            aggfunc='sum' )

for tm in df_sim.batting_team.unique():
    #print(tm)
    #tm = 'SRH'
    tm_df = runs_table[runs_table[tm].isna()==False][tm]>\
                     runs_table[runs_table[tm].isna()==False].sum(axis=1)/2

    tm_df = tm_df.reset_index()
    tm_df.columns= ['match_id','win_fl']
    df_name = tm + "_df"  

    # Assign the DataFrame to the dynamically created name
    globals()[df_name] = tm_df



In [ ]:
bat_tm_match_ov = df_sim.groupby(['batting_team','match_id','over_']).agg(
    inning = ('innings','unique'),
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
).reset_index()

bat_tm_match_ov['win_fl'] = False
for index,row in bat_tm_match_ov.iterrows():
    inning = row['inning'][0]
    tm = row['batting_team']
    m_id = row['match_id']
    df_name = tm + "_df"  
    df = globals()[df_name]
    win_fl = df[df.match_id==m_id].win_fl.unique()[0]
    
    bat_tm_match_ov.at[index,'win_fl'] = win_fl
    bat_tm_match_ov.at[index,'inning'] = inning

bat_tm_match_ov

In [ ]:
#impact overs --  more wickets => loss?

wkts = 3
bat_tm_match_ov[bat_tm_match_ov.outs>=wkts].drop_duplicates(['match_id','inning']).win_fl.mean(), \
        bat_tm_match_ov[bat_tm_match_ov.outs>=wkts].drop_duplicates(['match_id','inning']).shape[0]

## comment : Yepp

In [ ]:
#impact overs --  more runs => win?

runs = 20
bat_tm_match_ov[bat_tm_match_ov.runs>=runs].drop_duplicates(['match_id','inning']).win_fl.mean(), \
        bat_tm_match_ov[bat_tm_match_ov.runs>=runs].drop_duplicates(['match_id','inning']).shape[0]

## comment : Nope

In [287]:
tm = df_sim[df_sim.match_id.str.startswith('S02')].groupby('venue').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

tm['SR'] = 100*tm['runs']/tm['balls']
tm['Bat_avg'] = tm['runs']/tm['outs']
tm['bpb'] = tm['balls']/(tm['fours']+tm['sixes'])
tm.sort_values('Bat_avg',ascending=False)


,venue,runs,balls,outs,fours,sixes,SR,Bat_avg,bpb
1,DC,2870,1660,68,227,141,172.891566,42.205882,4.510870
9,SRH,2674,1658,66,199,119,161.278649,40.515152,5.213836
5,MI,2620,1629,66,221,107,160.834868,39.696970,4.966463
2,GT,2611,1634,68,210,95,159.791922,38.397059,5.357377
0,CSK,3280,2116,96,263,132,155.009452,34.166667,5.356962
7,RCB,2525,1631,77,205,104,154.812998,32.792208,5.278317
6,PBKS,3136,2101,98,239,132,149.262256,32.000000,5.663073
4,LSG,2519,1635,85,203,103,154.067278,29.635294,5.343137
8,RR,2485,1576,88,194,112,157.677665,28.238636,5.150327
3,KKR,2463,1621,90,215,101,151.943245,27.366667,5.129747


In [286]:
bowl_tm = df_sim[df_sim.match_id.str.startswith('S02')].groupby('venue').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

bowl_tm['Eco'] = 6*bowl_tm['runs']/bowl_tm['balls']
bowl_tm['bowl_SR'] = bowl_tm['balls']/bowl_tm['outs']
bowl_tm['bpb'] = bowl_tm['balls']/(bowl_tm['fours']+bowl_tm['sixes'])
bowl_tm.sort_values('Eco',ascending=True)

,venue,runs,balls,outs,fours,sixes,Eco,bowl_SR,bpb
6,PBKS,3136,2101,98,239,132,8.955735,21.438776,5.663073
3,KKR,2463,1621,90,215,101,9.116595,18.011111,5.129747
4,LSG,2519,1635,85,203,103,9.244037,19.235294,5.343137
7,RCB,2525,1631,77,205,104,9.288780,21.181818,5.278317
0,CSK,3280,2116,96,263,132,9.300567,22.041667,5.356962
8,RR,2485,1576,88,194,112,9.460660,17.909091,5.150327
2,GT,2611,1634,68,210,95,9.587515,24.029412,5.357377
5,MI,2620,1629,66,221,107,9.650092,24.681818,4.966463
9,SRH,2674,1658,66,199,119,9.676719,25.121212,5.213836
1,DC,2870,1660,68,227,141,10.373494,24.411765,4.510870


In [ ]:
bowl_tm.sort_values('venue',ascending=True)

## Powerplay

In [489]:
#powerplay bashers - Team

pp_bat_tm = df_sim[df_sim.match_id.str.startswith('S02')][df_sim.phase=='PP'].groupby('batting_team').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

pp_bat_tm['SR'] = 100*pp_bat_tm['runs']/pp_bat_tm['balls']
pp_bat_tm['Bat_avg'] = pp_bat_tm['runs']/pp_bat_tm['outs']
pp_bat_tm['bpb'] = pp_bat_tm['balls']/(pp_bat_tm['fours']+pp_bat_tm['sixes'])
pp_bat_tm.sort_values('SR',ascending=False).round(2)


## comment :: two teams went over 175 SR in PP, none qualified, BUT 3 teams went over avg 50, ALL 3 qualified 

,batting_team,runs,balls,outs,fours,sixes,SR,Bat_avg,bpb
9,SRH,1018,576,14,94,51,176.74,72.71,3.97
8,RR,1072,612,15,97,53,175.16,71.47,4.08
5,MI,881,540,17,104,24,163.15,51.82,4.22
1,DC,913,576,20,77,45,158.51,45.65,4.72
7,RCB,795,504,18,93,27,157.74,44.17,4.20
4,LSG,786,504,19,90,27,155.95,41.37,4.31
2,GT,778,504,20,81,26,154.37,38.90,4.71
6,PBKS,765,504,23,77,34,151.79,33.26,4.54
3,KKR,762,504,13,91,25,151.19,58.62,4.34
0,CSK,697,504,24,82,17,138.29,29.04,5.09


In [493]:
#powerplay stiflers - Team

pp_bowl_tm = df_sim[df_sim.match_id.str.startswith('S02')][df_sim.phase=='PP'].groupby('bowling_team').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

pp_bowl_tm['Eco'] = 6*pp_bowl_tm['runs']/pp_bowl_tm['balls']
pp_bowl_tm['bowl_SR'] = pp_bowl_tm['balls']/pp_bowl_tm['outs']
pp_bowl_tm['bpb'] = pp_bowl_tm['balls']/(pp_bowl_tm['fours']+pp_bowl_tm['sixes'])
pp_bowl_tm.sort_values('bowl_SR',ascending=True).round(2)


## comment :: 4 teams went SR <=20 in PP, ALL 4 qualified

,bowling_team,runs,balls,outs,fours,sixes,Eco,bowl_SR,bpb
4,LSG,794,504,22,79,36,9.45,22.91,4.38
3,KKR,813,504,21,91,30,9.68,24.00,4.17
2,GT,828,504,20,99,23,9.86,25.20,4.13
6,PBKS,769,504,20,88,25,9.15,25.20,4.46
9,SRH,917,576,21,83,43,9.55,27.43,4.57
0,CSK,794,504,17,85,30,9.45,29.65,4.38
5,MI,825,540,18,83,33,9.17,30.00,4.66
1,DC,935,576,18,100,35,9.74,32.00,4.27
7,RCB,803,504,13,66,38,9.56,38.77,4.85
8,RR,989,612,13,112,36,9.70,47.08,4.14


In [507]:
match_sum = match_sum[['Match_Num','Winner']]
match_sum['match_id'] = match_sum['Match_Num'].apply(lambda x: 'S02M0'+str(x) if x>9 else 'S02M00'+str(x))

#match_sum

In [511]:
pp_bat_tm_match = df_sim[df_sim.match_id.str.startswith('S02')]\
        [df_sim.phase=='PP'].groupby(['batting_team','match_id']).agg(
    inning = ('innings','unique'),
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

pp_bat_tm_match['SR'] = 100*pp_bat_tm_match['runs']/pp_bat_tm_match['balls']
#pp_bat_tm_match['Bat_avg'] = pp_bat_tm_match['runs']/pp_bat_tm_match['outs']
pp_bat_tm_match['bpb'] = pp_bat_tm_match['balls']/(pp_bat_tm_match['fours']+pp_bat_tm_match['sixes'])

pp_bat_tm_match['win_fl'] = 0
for index,row in pp_bat_tm_match.iterrows():
    tm = row['batting_team']
    m_id = row['match_id']
    df = match_sum[match_sum.match_id==m_id]
    winner = df['Winner'].unique()[0]
    if winner==tm:
        win_fl = 1
    else:
        win_fl = 0
    
    pp_bat_tm_match.at[index,'win_fl'] = win_fl

pp_bat_tm_match.sort_values('match_id')

In [518]:
# wickets => loss?

1-pp_bat_tm_match[pp_bat_tm_match.outs>=3].win_fl.mean(), pp_bat_tm_match[pp_bat_tm_match.outs>=3].shape[0]

## comment :: YEPP

(0.8571428571428572, 14)

In [523]:
# high SR => win?

pp_bat_tm_match[pp_bat_tm_match.SR>=225].win_fl.mean(), pp_bat_tm_match[pp_bat_tm_match.SR>=225].shape[0]

## comment ::  YEPP

(0.0, 1)

In [566]:
##powerplay bashers - Batters

pp_bat = df_sim[df_sim.match_id.str.startswith('S02')]\
        [df_sim.phase=="PP"].groupby('striker').agg(
    team = ('batting_team','unique'),
    runs = ('total_runs','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    dots = ('isDotforBatter','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

pp_bat['SR'] = 100*pp_bat['runs']/pp_bat['balls']
pp_bat['avg'] = pp_bat['runs']/pp_bat['outs']
pp_bat['bp_dismissal'] = pp_bat['balls']/pp_bat['outs']
pp_bat['bdry'] = pp_bat['fours']+pp_bat['sixes']
to_save = pp_bat[(pp_bat.balls>=50)&(pp_bat.bp_dismissal>=15)]#.sort_values('SR',ascending=False).head(15)

to_save.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Stats/pp_bat.csv', index=None)

## comment ::  considering 5 balls per game & 15 bpd, YBJ, Faf had the most SR

pp_bat[(pp_bat.balls>=50)&(pp_bat.bp_dismissal>=15)].sort_values('runs',ascending=False).head(15)

In [564]:
##powerplay stiflers - Bowlers

pp_bowl = df_sim[df_sim.match_id.str.startswith('S02')]\
        [df_sim.phase=="PP"].groupby('bowler').agg(
    team = ('bowling_team','unique'),
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    wkts = ('isBowlerWicket','sum'),
    dots = ('isDotforbowler','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

pp_bowl['Eco'] = 6*pp_bowl['runs']/pp_bowl['balls']
pp_bowl['bdry'] = pp_bowl['fours']+pp_bowl['sixes']
pp_bowl['SR'] = pp_bowl['balls']/pp_bowl['wkts']
to_save = pp_bowl[pp_bowl.balls>=30]#.sort_values('Eco',ascending=True).head(15)

to_save.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Stats/pp_bowl.csv', index=None)

## comment :: subj. to min. 10 overs, Farooqi & Kuldeep only went below 8.5 rpo.

In [565]:
pp_bowl.sort_values('wkts',ascending=False).round(2).head(7)

##comment ::  top wkt-takers in PP

,bowler,team,runs,balls,wkts,dots,fours,sixes,Eco,bdry,SR
54,MA Starc,[KKR],253,144,12,47,26,12,10.54,38,12.00
116,Washington Sundar,[PBKS],184,126,8,42,22,5,8.76,27,15.75
60,MP Yadav,[CSK],181,126,7,49,20,4,8.62,24,18.00
83,RA Jadeja,[DC],187,120,6,35,17,6,9.35,23,20.00
9,Arshdeep Singh,[LSG],123,96,6,42,12,5,7.69,17,16.00
37,JJ Bumrah,[MI],194,114,6,42,14,12,10.21,26,19.00
31,I Sharma,[GT],31,30,5,17,2,1,6.20,3,6.00


## Middle

In [528]:
#middle bashers - Team

middle_bat_tm = df_sim[df_sim.match_id.str.startswith('S02')]\
                        [df_sim.phase=='Middle'].groupby('batting_team').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

middle_bat_tm['SR'] = 100*middle_bat_tm['runs']/middle_bat_tm['balls']
middle_bat_tm['Bat_avg'] = middle_bat_tm['runs']/middle_bat_tm['outs']
middle_bat_tm['bpb'] = middle_bat_tm['balls']/(middle_bat_tm['fours']+middle_bat_tm['sixes'])
middle_bat_tm.sort_values('bpb',ascending=False).round(2)

## comment :: DC struck the best in middle overs, while GT had not lost wkts

,batting_team,runs,balls,outs,fours,sixes,SR,Bat_avg,bpb
0,CSK,1067,756,26,62,41,141.14,41.04,7.34
9,SRH,1238,864,28,81,37,143.29,44.21,7.32
4,LSG,1128,756,33,60,49,149.21,34.18,6.94
8,RR,1303,899,37,88,48,144.94,35.22,6.61
6,PBKS,1104,756,32,74,41,146.03,34.50,6.57
5,MI,1206,810,36,84,40,148.89,33.50,6.53
7,RCB,1152,756,24,68,48,152.38,48.00,6.52
3,KKR,1102,756,39,71,47,145.77,28.26,6.41
2,GT,1205,756,16,86,41,159.39,75.31,5.95
1,DC,1412,864,38,100,61,163.43,37.16,5.37


In [531]:
#middle stiflers - Team

middle_bowl_tm = df_sim[df_sim.match_id.str.startswith('S02')]\
                        [df_sim.phase=='Middle'].groupby('bowling_team').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

middle_bowl_tm['Eco'] = 6*middle_bowl_tm['runs']/middle_bowl_tm['balls']
middle_bowl_tm['bowl_SR'] = middle_bowl_tm['balls']/middle_bowl_tm['outs']
middle_bowl_tm['bpb'] = middle_bowl_tm['balls']/(middle_bowl_tm['fours']+middle_bowl_tm['sixes'])
middle_bowl_tm.sort_values('bpb',ascending=True).round(2)

## comment :: GT,MI,PBKS went <9 rpo, while DC had the best SR <15 in middle overs

,bowling_team,runs,balls,outs,fours,sixes,Eco,bowl_SR,bpb
9,SRH,1330,864,40,81,64,9.24,21.60,5.96
2,GT,1133,756,37,88,37,8.99,20.43,6.05
7,RCB,1142,756,32,78,46,9.06,23.62,6.10
1,DC,1316,864,25,85,50,9.14,34.56,6.40
5,MI,1179,791,24,80,42,8.94,32.96,6.48
8,RR,1403,918,38,89,52,9.17,24.16,6.51
4,LSG,1124,756,29,76,40,8.92,26.07,6.52
3,KKR,1129,756,32,65,48,8.96,23.62,6.69
6,PBKS,1057,756,29,68,38,8.39,26.07,7.13
0,CSK,1104,756,23,64,36,8.76,32.87,7.56


In [534]:
middle_bat_tm_match = df_sim[df_sim.match_id.str.startswith('S02')]\
            [df_sim.phase=='Middle'].groupby(['batting_team','match_id']).agg(
    inning = ('innings','unique'),
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

middle_bat_tm_match['SR'] = 100*middle_bat_tm_match['runs']/middle_bat_tm_match['balls']
middle_bat_tm_match['bpb'] = middle_bat_tm_match['balls']/(middle_bat_tm_match['fours']+middle_bat_tm_match['sixes'])


middle_bat_tm_match[middle_bat_tm_match.batting_team=='CSK'].sort_values('SR',ascending=False)

middle_bat_tm_match['win_fl'] = 0
for index,row in middle_bat_tm_match.iterrows():
    tm = row['batting_team']
    m_id = row['match_id']
    df = match_sum[match_sum.match_id==m_id]
    winner = df['Winner'].unique()[0]
    if winner==tm:
        win_fl = 1
    else:
        win_fl = 0
    
    middle_bat_tm_match.at[index,'win_fl'] = win_fl

In [540]:
# wickets => loss?

wkts=4
1-middle_bat_tm_match[middle_bat_tm_match.outs>=wkts].win_fl.mean(), \
            middle_bat_tm_match[middle_bat_tm_match.outs>=wkts].shape[0]

## comment ::  YEPP

(0.7083333333333333, 24)

In [541]:
# high SR => win?

SR = 175
middle_bat_tm_match[middle_bat_tm_match.SR>=SR].win_fl.mean(),\
            middle_bat_tm_match[middle_bat_tm_match.SR>=SR].shape[0]

## comment ::  Nope

(0.4, 15)

In [563]:
##Middle bashers - Batters

middle_bat = df_sim[df_sim.match_id.str.startswith('S02')]\
        [df_sim.phase=="Middle"].groupby('striker').agg(
    team = ('batting_team','unique'),
    runs = ('total_runs','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    dots = ('isDotforBatter','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

middle_bat['SR'] = 100*middle_bat['runs']/middle_bat['balls']
middle_bat['bp_dismissal'] = middle_bat['balls']/middle_bat['outs']
middle_bat['dot%'] = 100*middle_bat['dots']/middle_bat['balls']
middle_bat['bdry'] = middle_bat['fours']+middle_bat['sixes']
to_save = middle_bat[(middle_bat.balls>=50)&(middle_bat.bp_dismissal>=20)]#.sort_values('SR',ascending=False).head(15)

to_save.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Stats/middle_bat.csv', index=None)
## comment ::  batters with most SR v dot% in middle, subj. to balls>=50, bpd>=20

In [ ]:
middle_bat[middle_bat.balls>=50].sort_values('bp_dismissal',ascending=False).head(15)

In [561]:
##middle wkt-takers - Bowlers

middle_bowl = df_sim[df_sim.match_id.str.startswith('S02')]\
            [df_sim.phase=="Middle"].groupby('bowler').agg(
    team = ('bowling_team','unique'),
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    wkts = ('isBowlerWicket','sum'),
    dots = ('isDotforbowler','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

middle_bowl['Eco'] = 6*middle_bowl['runs']/middle_bowl['balls']
middle_bowl['bdry'] = middle_bowl['fours']+middle_bowl['sixes']
middle_bowl['SR'] = middle_bowl['balls']/middle_bowl['wkts']
to_save = middle_bowl[middle_bowl.balls>=42]#.sort_values('SR',ascending=True).head(15)

to_save.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Stats/middle_bowl.csv', index=None)
## comment :: subj. to min 10 overs, Ghazanfar picked the most wkts & best SR! plot SR v Eco.

In [562]:
middle_bowl.sort_values('wkts',ascending=False).head(15)

,bowler,team,runs,balls,wkts,dots,fours,sixes,Eco,bdry,SR
51,M Pathirana,[RCB],203,138,12,45,14,9,8.826087,23,11.500000
63,Mayank Dagar,[RR],214,174,10,55,12,6,7.379310,18,17.400000
30,Harpreet Brar,[RCB],207,138,9,35,14,8,9.000000,22,15.333333
14,B Kumar,[KKR],244,144,9,36,18,11,10.166667,29,16.000000
94,SM Curran,[KKR],200,138,8,39,10,11,8.695652,21,17.250000
88,Rasikh Salam,[GT],116,78,8,22,11,3,8.923077,14,9.750000
38,JJ Bumrah,[MI],189,144,8,36,11,5,7.875000,16,18.000000
87,Rashid Khan,[GT],255,168,7,36,22,6,9.107143,28,24.000000
5,AK Markram,[SRH],119,78,7,24,6,7,9.153846,13,11.142857
52,M Prasidh Krishna,[DC],283,216,7,53,19,5,7.861111,24,30.857143


## Death

In [545]:
#death bashers - Team

death_bat_tm = df_sim[df_sim.match_id.str.startswith('S02')]\
            [df_sim.phase=='Death'].groupby('batting_team').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

death_bat_tm['SR'] = 100*death_bat_tm['runs']/death_bat_tm['balls']
death_bat_tm['Bat_avg'] = death_bat_tm['runs']/death_bat_tm['outs']
death_bat_tm['bpb'] = death_bat_tm['balls']/(death_bat_tm['fours']+death_bat_tm['sixes'])
death_bat_tm.sort_values('Bat_avg',ascending=False).round(2)

## comment :: DC struck the best in death overs, while GT had not lost wkts

,batting_team,runs,balls,outs,fours,sixes,SR,Bat_avg,bpb
1,DC,846,460,24,68,41,183.91,35.25,4.22
9,SRH,785,455,26,66,34,172.53,30.19,4.55
5,MI,675,370,25,48,42,182.43,27.00,4.11
0,CSK,716,405,27,70,32,176.79,26.52,3.97
8,RR,815,437,35,61,49,186.50,23.29,3.97
4,LSG,591,356,29,37,38,166.01,20.38,4.75
7,RCB,635,373,33,48,35,170.24,19.24,4.49
6,PBKS,568,374,35,40,28,151.87,16.23,5.50
2,GT,577,362,36,30,32,159.39,16.03,5.84
3,KKR,591,368,40,48,33,160.60,14.78,4.54


In [548]:
#death stiflers - Team

death_bowl_tm = df_sim[df_sim.match_id.str.startswith('S02')]\
                    [df_sim.phase=='Death'].groupby('bowling_team').agg(
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

death_bowl_tm['Eco'] = 6*death_bowl_tm['runs']/death_bowl_tm['balls']
death_bowl_tm['bowl_SR'] = death_bowl_tm['balls']/death_bowl_tm['outs']
death_bowl_tm['bpb'] = death_bowl_tm['balls']/(death_bowl_tm['fours']+death_bowl_tm['sixes'])
death_bowl_tm.sort_values('Eco',ascending=True).round(2)

## comment :: GT,MI,PBKS went <9 rpo, while DC had the best SR <15 in death overs

,bowling_team,runs,balls,outs,fours,sixes,Eco,bowl_SR,bpb
0,CSK,585,366,31,41,34,9.59,11.81,4.88
6,PBKS,577,360,27,40,32,9.62,13.33,5.00
9,SRH,713,443,42,55,37,9.66,10.55,4.82
7,RCB,617,378,24,46,31,9.79,15.75,4.91
4,LSG,612,374,37,57,29,9.82,10.11,4.35
2,GT,586,348,30,39,38,10.10,11.60,4.52
8,RR,820,457,37,59,44,10.77,12.35,4.44
1,DC,851,462,35,69,39,11.05,13.20,4.28
3,KKR,687,372,25,51,39,11.08,14.88,4.13
5,MI,751,400,22,59,41,11.26,18.18,4.00


In [555]:
##death bashers - Batters

death_bat = df_sim[df_sim.match_id.str.startswith('S02')]\
                [df_sim.phase=="Death"].groupby('striker').agg(
    team = ('batting_team','unique'),
    runs = ('total_runs','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('is_striker_Out','sum'),
    dots = ('isDotforBatter','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

death_bat['SR'] = 100*death_bat['runs']/death_bat['balls']
death_bat['bp_dismissal'] = death_bat['balls']/death_bat['outs']
death_bat['dot%'] = 100*death_bat['dots']/death_bat['balls']
death_bat['bdry'] = death_bat['fours']+death_bat['sixes']
to_save = death_bat[(death_bat.balls>=35)&(death_bat.bp_dismissal>=10)]#.sort_values('SR',ascending=False).head(15)

to_save.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Stats/death_bat.csv', index=None)
## comment ::  batters with most SR v dot% in death, subj. to balls>=50, bpd>=20

In [554]:
death_bat[death_bat.balls>=35].sort_values('SR',ascending=False).round(2).head(15)

,striker,team,runs,balls,outs,dots,fours,sixes,SR,bp_dismissal,dot%,bdry
36,GJ Maxwell,[RR],101,42,2,3,4,9,240.48,21.00,7.14,13
154,Tilak Varma,[CSK],83,35,2,11,6,7,237.14,17.50,31.43,13
55,Jay Gohil,[MI],114,49,2,14,7,10,232.65,24.50,28.57,17
119,RR Rossouw,[MI],151,65,6,18,10,11,232.31,10.83,27.69,21
162,YBK Jaiswal,[RR],249,110,4,17,19,16,226.36,27.50,15.45,35
142,Shashank Singh,[PBKS],124,55,5,11,9,8,225.45,11.00,20.00,17
149,T Stubbs,[CSK],105,47,2,12,7,7,223.40,23.50,25.53,14
120,Ramandeep Singh,[LSG],84,38,2,9,3,8,221.05,19.00,23.68,11
125,S Dube,[RR],201,91,6,18,18,11,220.88,15.17,19.78,29
3,AM Rahane,[KKR],115,53,5,16,13,6,216.98,10.60,30.19,19


In [560]:
##death wkt-takers - Bowlers

death_bowl = df_sim[df_sim.match_id.str.startswith('S02')]\
        [df_sim.phase=="Death"].groupby('bowler').agg(
    team = ('bowling_team','unique'),
    runs = ('total_runs','sum'),
    balls = ('islegal','sum'),
    wkts = ('isBowlerWicket','sum'),
    dots = ('isDotforbowler','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

death_bowl['Eco'] = 6*death_bowl['runs']/death_bowl['balls']
death_bowl['bdry'] = death_bowl['fours']+death_bowl['sixes']
death_bowl['SR'] = death_bowl['balls']/death_bowl['wkts']
to_save= death_bowl[death_bowl.balls>=30]#.sort_values('Eco',ascending=True).head(15)

to_save.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Stats/death_bowl.csv', index=None)
## comment :: subj. to min 10 overs, Ghazanfar picked the most wkts & best SR! plot SR v Eco.

In [559]:
death_bowl.sort_values('wkts',ascending=False).head(15)

,bowler,team,runs,balls,wkts,dots,fours,sixes,Eco,bdry,SR
34,JC Archer,[LSG],169,110,13,31,11,9,9.218182,20,8.461538
58,MP Yadav,[CSK],178,128,12,56,13,10,8.343750,23,10.666667
4,AK Markram,[SRH],141,90,9,35,11,9,9.400000,20,10.000000
62,Mohammad Nabi,[GT],111,72,9,27,8,7,9.250000,15,8.000000
47,LS Livingstone,[SRH],128,73,9,23,9,7,10.520548,16,8.111111
13,B Kumar,[KKR],174,91,8,24,12,11,11.472527,23,11.375000
68,Mustafizur Rahman,[DC],107,84,7,26,7,4,7.642857,11,12.000000
2,A Zampa,[SRH],62,62,7,27,3,2,6.000000,5,8.857143
54,MA Starc,[KKR],182,102,7,29,11,12,10.705882,23,14.571429
81,RA Jadeja,[DC],145,78,7,21,11,9,11.153846,20,11.142857


## best partnership

In [ ]:
#total_runs, is_faced_by_batter

In [266]:
df_sim['partners'] = ''
for index,row in df_sim.iterrows():
    bat_1 = row['striker']
    bat_2 = row['non_striker']
    partners = str(set([bat_1,bat_2]))
    df_sim.loc[index,'partners'] = partners
    

In [268]:
partnership = df_sim[df_sim.match_id.str.startswith('S02')].groupby('partners').agg(  ## in s02 only
    runs = ('total_runs','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum')
).reset_index()

ball = 100

partnership['SR'] = 100*partnership['runs']/partnership['balls']
partnership['Bat_avg'] = partnership['runs']/partnership['outs']
partnership['bpb'] = partnership['balls']/(partnership['fours']+partnership['sixes'])

In [269]:
partnership[partnership['balls']>=ball].sort_values(['SR','Bat_avg'],ascending=False)

,partners,runs,balls,outs,fours,sixes,SR,Bat_avg,bpb
348,"{'S Dube', 'YBK Jaiswal'}",397,203,5,28,20,195.566502,79.400000,4.229167
127,"{'GJ Maxwell', 'YBK Jaiswal'}",250,136,3,19,12,183.823529,83.333333,4.387097
124,"{'GJ Maxwell', 'RD Rickelton'}",833,457,14,67,41,182.275711,59.500000,4.231481
421,"{'V Kohli', 'Abishek Porel'}",218,122,4,18,8,178.688525,54.500000,4.692308
126,"{'GJ Maxwell', 'S Dube'}",184,105,7,10,14,175.238095,26.285714,4.375000
236,"{'N Pooran', 'RR Rossouw'}",289,166,6,24,10,174.096386,48.166667,4.882353
275,"{'PP Shaw', 'Abhishek Sharma'}",409,236,10,43,12,173.305085,40.900000,4.290909
199,"{'KL Rahul', 'BM Duckett'}",404,234,6,32,15,172.649573,67.333333,4.978723
266,"{'PD Salt', 'RD Gaikwad'}",707,410,13,75,27,172.439024,54.384615,4.019608
425,"{'V Kohli', 'SA Yadav'}",417,243,10,32,18,171.604938,41.700000,4.860000


## best rivals

In [ ]:
#runs_off_bat, is_faced_by_batter

df_sim.columns

In [273]:
rivalry = df_sim[df_sim.match_id.str.startswith('S02')].groupby(['striker','bowler']).agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum')
).reset_index()

ball = 15

rivalry['SR'] = 100*rivalry['runs']/rivalry['balls']
rivalry['Bat_avg'] = rivalry['runs']/rivalry['outs']
rivalry['bpb'] = rivalry['balls']/(rivalry['fours']+rivalry['sixes'])
rivalry['dot_%'] = 100*rivalry['dots']/rivalry['balls']

In [274]:
rivalry[rivalry['balls']>=ball].sort_values(['SR','Bat_avg'],ascending=False)

,striker,bowler,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%
2649,Smaran Ravichandran,YS Chahal,48,15,0,3,5,2,320.000000,inf,1.875000,13.333333
3020,YBK Jaiswal,HH Pandya,56,19,1,6,4,2,294.736842,56.0,1.900000,10.526316
694,GJ Maxwell,MR Marsh,44,15,0,0,5,2,293.333333,inf,3.000000,13.333333
2280,SA Yadav,Suyash Sharma,58,23,0,3,5,1,252.173913,inf,2.875000,4.347826
418,BM Duckett,Noor Ahmad,40,16,0,3,3,2,250.000000,inf,2.666667,12.500000
1335,N Pooran,M Pathirana,44,19,0,7,1,2,231.578947,inf,2.375000,10.526316
856,J Fraser-McGurk,SM Curran,37,16,1,5,2,4,231.250000,37.0,2.285714,25.000000
2846,V Kohli,A Kamboj,34,15,0,5,1,2,226.666667,inf,2.500000,13.333333
698,GJ Maxwell,Mustafizur Rahman,47,21,0,3,4,3,223.809524,inf,3.000000,14.285714
1939,RD Rickelton,Mustafizur Rahman,33,15,1,2,3,3,220.000000,33.0,3.000000,20.000000


In [275]:
rivalry[rivalry['balls']>=ball].sort_values(['outs','SR'],ascending=[False,True])

,striker,bowler,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%
1652,PP Shaw,JR Hazlewood,12,16,2,1,0,8,75.000000,6.0,16.000000,50.000000
474,D Brevis,GJ Maxwell,23,16,2,3,1,7,143.750000,11.5,4.000000,43.750000
1559,P Simran Singh,Sandeep Sharma,43,28,2,3,1,6,153.571429,21.5,7.000000,21.428571
598,F du Plessis,DS Rathi,14,15,1,1,0,6,93.333333,14.0,15.000000,40.000000
814,Ishan Kishan,Arshdeep Singh,16,16,1,1,1,9,100.000000,16.0,8.000000,56.250000
1976,RG Sharma,Kuldeep Yadav,15,15,1,3,0,10,100.000000,15.0,5.000000,66.666667
2053,RM Patidar,Akash Deep,16,16,1,1,0,6,100.000000,16.0,16.000000,37.500000
2892,V Kohli,Sandeep Sharma,15,15,1,0,1,5,100.000000,15.0,15.000000,33.333333
1739,Q de Kock,Sikandar Raza,18,17,1,1,1,9,105.882353,18.0,8.500000,52.941176
1742,Q de Kock,T Natarajan,16,15,1,2,0,6,106.666667,16.0,7.500000,40.000000
